# **Importing Libraries**

First, All the necessary libraries are to be imported 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import read_csv
import math
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import TimeSeriesSplit
from keras.callbacks import LearningRateScheduler
from sklearn.model_selection import GridSearchCV
from keras.constraints import maxnorm
import joblib
from numpy.random import seed
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

Setting Random seed value

In [3]:
seed(1)
tf.random.set_seed=(1)

# **Define number of variables**

For Input combination 1, variables chosen are Tp and WL;         
For Input combination 2, variables chosen are Tp, Swvl1,2,3 and WL;         
For Input combination 3, variables chosen are Tp, Swvl1,2,3, Tcrw and WL;       
For Input combination 4, variables chosen are Tp, Swvl1,2,3, Tcrw, Tcwv and WL

In [4]:
comb=1


In [5]:
if comb==1: var,vars=2,"Tp"
elif comb==2: var,vars=5,"TpSwvl"
elif comb==3: var,vars=6,"TpSwvlTcrw"
elif comb==4: var,vars=7,"TpSwvlTcrw"

print("Input Combination: ",comb)
print("number of variables: ", var)

Input Combination:  1
number of variables:  2


# **Data Import**

First, we must mount the colab file to the drive containing our data. This will require google sign in

For Google sign-in please use the following credentials:

email: ***bscthesis016@gmail.com***

Password: ***Thesis016***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataframe = pd.read_excel("/content/drive/MyDrive/Thesis/data.xlsx")
#df = pd.read_excel("data.xlsx")
dataframe

# **Data Processing For ANN, Random Forest, CatBoost**

Now we will have to process the data for using in our models. We start by setting the dates as index

In [ ]:
df=dataframe.set_index('Date')

Then we select our list of variables by columns from the dataframe.

In [ ]:
cols = list(df)[0:var]
print(cols)

now we create dataframe df containing only the variables to be used

In [ ]:
df = df[cols].astype(float) #Convert values to float
df

**Declaraion: All variable names from here onwards will contain a number. the number is indicative of the lead time**

Now, for different lead times, we prepare the input and target dataset

In [ ]:
dfX1 = df[:-1]
dfX3 = df[:-3]
dfX5 = df[:-5]
dfX7 = df[:-7]

Now we see that our input dataset for 7 day lead spans from 1/1/2007 to 24/12/2020

In [ ]:
dfX7

Checking to see if the remaining input datasets are in proper shape

In [ ]:
print("Shape of dfX1: {}".format(dfX1.shape))
print("Shape of dfX3: {}".format(dfX3.shape))
print("Shape of dfX5: {}".format(dfX5.shape))
print("Shape of dfX7: {}".format(dfX7.shape))

Repeating the procedure for our target dataset we get:

In [ ]:
df_Y=df[['WL']]
dfY1=df_Y[1:]
dfY3=df_Y[3:]
dfY5=df_Y[5:]
dfY7=df_Y[7:]

Now we see that the output dataset for 7day lead spans from 8/1/2007 to 31/12/2020

In [ ]:
dfY7

Checking the shapes for remaining lead times:

In [ ]:
print("Shape of dfY1: {}".format(dfY1.shape))
print("Shape of dfY3: {}".format(dfY3.shape))
print("Shape of dfY5: {}".format(dfY5.shape))
print("Shape of dfY7: {}".format(dfY7.shape))

Now we split the datasets into training, validation and testing sets, with 50%, 20% and 30% data respectively 

In [ ]:
train_size1 = int(len(dfX1) * 0.5)
val_size1 = int(len(dfX1) * 0.2)
test_size1 = len(dfX1) - train_size1 -val_size1
trainX1, valX1, testX1 = dfX1[0:train_size1], dfX1[train_size1:-test_size1], dfX1[-test_size1:]
trainY1, valY1, testY1 = dfY1[0:train_size1], dfY1[train_size1:-test_size1], dfY1[-test_size1:]

train_size3 = int(len(dfX3) * 0.5)
val_size3 = int(len(dfX3) * 0.2)
test_size3 = len(dfX3) - train_size3 -val_size3
trainX3, valX3, testX3 = dfX3[0:train_size3], dfX3[train_size3:-test_size3], dfX3[-test_size3:]
trainY3, valY3, testY3 = dfY3[0:train_size3], dfY3[train_size3:-test_size3], dfY3[-test_size3:]

train_size5 = int(len(dfX5) * 0.5)
val_size5 = int(len(dfX5) * 0.2)
test_size5 = len(dfX5) - train_size5 -val_size5
trainX5, valX5, testX5 = dfX5[0:train_size5], dfX5[train_size5:-test_size5], dfX5[-test_size5:]
trainY5, valY5, testY5 = dfY5[0:train_size5], dfY5[train_size5:-test_size5], dfY5[-test_size5:]

train_size7 = int(len(dfX7) * 0.5)
val_size7 = int(len(dfX7) * 0.2)
test_size7 = len(dfX7) - train_size7 -val_size7
trainX7, valX7, testX7 = dfX7[0:train_size7], dfX7[train_size7:-test_size7], dfX7[-test_size7:]
trainY7, valY7, testY7 = dfY7[0:train_size7], dfY7[train_size7:-test_size7], dfY7[-test_size7:]

Now we check the shape of train and test size samples for all lead times

In [ ]:
print("Shape of training set X1: {}".format(trainX1.shape))
print("Shape of test set X1: {}".format(testX1.shape))
print("Shape of training set Y1: {}".format(trainY1.shape))
print("Shape of test set Y1: {}".format(testY1.shape))

print("Shape of training set X3: {}".format(trainX3.shape))
print("Shape of test set X3: {}".format(testX3.shape))
print("Shape of training set Y3: {}".format(trainY3.shape))
print("Shape of test set Y3: {}".format(testY3.shape))

print("Shape of training set X5: {}".format(trainX5.shape))
print("Shape of test set X5: {}".format(testX5.shape))
print("Shape of training set Y5: {}".format(trainY5.shape))
print("Shape of test set Y5: {}".format(testY5.shape))

print("Shape of training set X7: {}".format(trainX7.shape))
print("Shape of test set X7: {}".format(testX7.shape))
print("Shape of training set Y7: {}".format(trainY7.shape))
print("Shape of test set Y7: {}".format(testY7.shape))




***Data Processing complete, now moving on to model building***

# **ANN**

If the models have already been develpoed, We can just import the saved models and move on to prediction. If not, then we have to train the models

## **Import Models**

we import 4 models for 4 different lead times

In [ ]:
ANN1 = load_model('/content/drive/MyDrive/Thesis/SavedModels/"+vars+"/ANN/ANN1.h5')

ANN3 = load_model('/content/drive/MyDrive/Thesis/SavedModels/"+vars+"/ANN/ANN3.h5')

ANN5 = load_model('/content/drive/MyDrive/Thesis/SavedModels/"+vars+"/ANN/ANN5.h5')

ANN7 = load_model('/content/drive/MyDrive/Thesis/SavedModels/"+vars+"/ANN/ANN7.h5')

## **BUilding models**

For building the DNN models, we define the model as sequential. then we add 6 dense layers with 64 neurons each, with activation function set to ReLU. the first layer contains the input variables and has the shape as the variables. The last dense layer was set with a single output.

finally, the models were compiled with loss function set to rmse and optimizer set to adam.

In [ ]:
#ANN model for 1 day Lead

print('Build deep model...')

ANN1 = Sequential()
ANN1.add(Dense(64, input_shape=(var,), activation='relu')) #12
#ANN1.add(Dropout(0.2))
ANN1.add(Dense(64, activation='relu'))  #8
#ANN1.add(Dropout(0.2))
ANN1.add(Dense(64,activation='relu'))
ANN1.add(Dense(64,activation='relu'))
ANN1.add(Dense(64,activation='relu'))
ANN1.add(Dense(64,activation='relu'))
ANN1.add(Dense(1))
ANN1.compile(loss='mean_squared_error', optimizer='Adam', metrics = [tf.keras.metrics.MeanSquaredError()])
print(ANN1.summary())

The same was repeated for all lead times.

In [ ]:
#ANN model for 3 day Lead

print('Build deep model...')

ANN3 = Sequential()
ANN3.add(Dense(64, input_shape=(var,), activation='relu'))
#ANN3.add(Dropout(0.2))
ANN3.add(Dense(64, activation='relu')) 
#ANN3.add(Dropout(0.2))
ANN3.add(Dense(64,activation='relu'))
ANN3.add(Dense(64,activation='relu'))
ANN3.add(Dense(64,activation='relu'))
ANN3.add(Dense(64,activation='relu'))
ANN3.add(Dense(1))
ANN3.compile(loss='mean_squared_error', optimizer='Adam', metrics = [tf.keras.metrics.MeanSquaredError()])
print(ANN3.summary())

In [ ]:
#ANN model for 5 day Lead

print('Build deep model...')

ANN5 = Sequential()
ANN5.add(Dense(64, input_shape=(var,), activation='relu')) 
#ANN5.add(Dropout(0.2))
ANN5.add(Dense(64, activation='relu')) 
#ANN5.add(Dropout(0.2))
ANN5.add(Dense(64,activation='relu'))
ANN5.add(Dense(64,activation='relu'))
ANN5.add(Dense(64,activation='relu'))
ANN5.add(Dense(64,activation='relu'))
ANN5.add(Dense(1))
ANN5.compile(loss='mean_squared_error', optimizer='Adam', metrics = [tf.keras.metrics.MeanSquaredError()])
print(ANN5.summary())

In [ ]:
#ANN model for 7 day Lead

print('Build deep model...')

ANN7 = Sequential()
ANN7.add(Dense(64, input_shape=(var,), activation='relu'))
#ANN7.add(Dropout(0.2))
ANN7.add(Dense(64, activation='relu'))
#ANN7.add(Dropout(0.2))
ANN7.add(Dense(64,activation='relu'))
ANN7.add(Dense(64,activation='relu'))
ANN7.add(Dense(64,activation='relu'))
ANN7.add(Dense(64,activation='relu'))
ANN7.add(Dense(1))
ANN7.compile(loss='mean_squared_error', optimizer='Adam', metrics = [tf.keras.metrics.MeanSquaredError()])
print(ANN7.summary())

The models are now ready to be trained

## **Model Train**

First, we define Earlystopping as a callback with patience set to 7. this means that training will stop when validation error does not reduce for 7 epochs. 

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=7)

With a batch size set to 64, callback set to our defined callback, and validation data set to our defined validation data split, we train the models for upto 200 epochs. Training may stop long before that thanks to earlystopping calllback

In [ ]:
history1 = ANN1.fit(trainX1, trainY1,  validation_data=[valX1, valY1],batch_size=64,callbacks=[callback],
          verbose=2, epochs=200)

We repeat the same process for all the time leads

In [ ]:
history3 = ANN3.fit(trainX3, trainY3,  validation_data=[valX3, valY3],batch_size=64,callbacks=[callback],
          verbose=2, epochs=200)

In [ ]:
history5 = ANN5.fit(trainX5, trainY5,  validation_data=[valX5, valY5],batch_size=64,callbacks=[callback],
          verbose=2, epochs=200)

In [ ]:
history7 = ANN7.fit(trainX7, trainY7,  validation_data=[valX7, valY7],batch_size=64,callbacks=[callback],
          verbose=2, epochs=200)

Training is complete.

## **Plots**

These plots show the change of training loss and validation loss while training the model.

In [ ]:
#plot the training and validation accuracy and loss at each epoch
train_loss1 = history1.history['loss']
val_loss1 = history1.history['val_loss']
epochs = range(1, len(train_loss1) + 1)
plt.plot(epochs, train_loss1, 'y', label='Training loss')
plt.plot(epochs, val_loss1, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
train_loss3 = history3.history['loss']
val_loss3 = history3.history['val_loss']
epochs = range(1, len(train_loss3) + 1)
plt.plot(epochs, train_loss3, 'y', label='Training loss')
plt.plot(epochs, val_loss3, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
train_loss5 = history5.history['loss']
val_loss5 = history5.history['val_loss']
epochs = range(1, len(train_loss5) + 1)
plt.plot(epochs, train_loss5, 'y', label='Training loss')
plt.plot(epochs, val_loss5, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
train_loss7 = history7.history['loss']
val_loss7 = history7.history['val_loss']
epochs = range(1, len(train_loss7) + 1)
plt.plot(epochs, train_loss7, 'y', label='Training loss')
plt.plot(epochs, val_loss7, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

## **prediction**

It's time to make some predictions using our held out test dataset. we also use the train and validation dataset to calculate trianing and validation errors

In [ ]:
trainpredictANN1 = ANN1.predict(trainX1)
valpredictANN1 = ANN1.predict(valX1)
testpredictANN1 = ANN1.predict(testX1)

trainpredictANN3 = ANN3.predict(trainX3)
valpredictANN3 = ANN3.predict(valX3)
testpredictANN3 = ANN3.predict(testX3)

trainpredictANN5 = ANN5.predict(trainX5)
valpredictANN5 = ANN5.predict(valX5)
testpredictANN5 = ANN5.predict(testX5)

trainpredictANN7 = ANN7.predict(trainX7)
valpredictANN7 = ANN7.predict(valX7)
testpredictANN7 = ANN7.predict(testX7)

## **Evaluation**

Now we calculate RMSE values for train, test and validation datasets

In [ ]:
trainScoreANN1 = math.sqrt(mean_squared_error(trainY1, trainpredictANN1))
print('Train Score 1: %.2f RMSE' % (trainScoreANN1))
valScoreANN1 = math.sqrt(mean_squared_error(valY1, valpredictANN1))
print('val Score 1: %.2f RMSE' % (valScoreANN1))
testScoreANN1 = math.sqrt(mean_squared_error(testY1, testpredictANN1))
print('Test Score 1: %.2f RMSE' % (testScoreANN1))

trainScoreANN3 = math.sqrt(mean_squared_error(trainY3, trainpredictANN3))
print('Train Score 3: %.2f RMSE' % (trainScoreANN3))
valScoreANN3 = math.sqrt(mean_squared_error(valY3, valpredictANN3))
print('val Score 3: %.2f RMSE' % (valScoreANN3))
testScoreANN3 = math.sqrt(mean_squared_error(testY3, testpredictANN3))
print('Test Score 3: %.2f RMSE' % (testScoreANN3))

trainScoreANN5 = math.sqrt(mean_squared_error(trainY5, trainpredictANN5))
print('Train Score 5: %.2f RMSE' % (trainScoreANN5))
valScoreANN5 = math.sqrt(mean_squared_error(valY5, valpredictANN5))
print('val Score 5: %.2f RMSE' % (valScoreANN5))
testScoreANN5 = math.sqrt(mean_squared_error(testY5, testpredictANN5))
print('Test Score 5: %.2f RMSE' % (testScoreANN5))

trainScoreANN7 = math.sqrt(mean_squared_error(trainY7, trainpredictANN7))
print('Train Score 7: %.2f RMSE' % (trainScoreANN7))
valScoreANN7 = math.sqrt(mean_squared_error(valY7, valpredictANN7))
print('val Score 7: %.2f RMSE' % (valScoreANN7))
testScoreANN7 = math.sqrt(mean_squared_error(testY7, testpredictANN7))
print('Test Score 7: %.2f RMSE' % (testScoreANN7))

Now we calculate r2 values for train, test and validation datasets

In [ ]:
trainr2_ANN1 =r2_score(trainY1, trainpredictANN1)
print('Train Score 1: %.2f RMSE' % (trainr2_ANN1))
valr2_ANN1 =r2_score(valY1, valpredictANN1)
print('val Score 1: %.2f RMSE' % (valr2_ANN1))
testr2_ANN1 =r2_score(testY1, testpredictANN1)
print('Test Score 1: %.2f RMSE' % (testr2_ANN1))

trainr2_ANN3 =r2_score(trainY3, trainpredictANN3)
print('Train Score 3: %.2f RMSE' % (trainr2_ANN3))
valr2_ANN3 =r2_score(valY3, valpredictANN3)
print('val Score 3: %.2f RMSE' % (valr2_ANN3))
testr2_ANN3 =r2_score(testY3, testpredictANN3)
print('Test Score 3: %.2f RMSE' % (testr2_ANN3))

trainr2_ANN5 =r2_score(trainY5, trainpredictANN5)
print('Train Score 5: %.2f RMSE' % (trainr2_ANN5))
valr2_ANN5 =r2_score(valY5, valpredictANN5)
print('val Score 5: %.2f RMSE' % (valr2_ANN5))
testr2_ANN5 =r2_score(testY5, testpredictANN5)
print('Test Score 5: %.2f RMSE' % (testr2_ANN5))

trainr2_ANN7 =r2_score(trainY7, trainpredictANN7)
print('Train Score 7: %.2f RMSE' % (trainr2_ANN7))
valr2_ANN7 =r2_score(valY7, valpredictANN7)
print('val Score 7: %.2f RMSE' % (valr2_ANN7))
testr2_ANN7 =r2_score(testY7, testpredictANN7)
print('Test Score 7: %.2f RMSE' % (testr2_ANN7))

Now we create a dataframe named rmse_vals including the rmse values for the training, validation and test datasets for the ANN model

In [ ]:
rmse_vals={'ANN':[trainScoreANN1,valScoreANN1,testScoreANN1,trainScoreANN3,valScoreANN3,testScoreANN3,trainScoreANN5,valScoreANN5,testScoreANN5,trainScoreANN7,valScoreANN7,testScoreANN7]}
df_rmse=pd.DataFrame(rmse_vals)

Now we create a dataframe named r2_vals including the r2 values for the training, validation and test datasets for the ANN model

In [ ]:
r2_vals={'ANN':[trainr2_ANN1,valr2_ANN1,testr2_ANN1,trainr2_ANN3,valr2_ANN3,testr2_ANN3,trainr2_ANN5,valr2_ANN5,testr2_ANN5,trainr2_ANN7,valr2_ANN7,testr2_ANN7]}
df_r2=pd.DataFrame(r2_vals)

## **Save ANN Models**

Time to save the models to path so that they can be used later.

In [ ]:
import os.path

In [ ]:
ANN1.save('/content/drive/MyDrive/Thesis/SavedModels/'+vars+'/ANN/ANN1.h5')

ANN3.save('/content/drive/MyDrive/Thesis/SavedModels/'+vars+'/ANN/ANN3.h5')

ANN5.save('/content/drive/MyDrive/Thesis/SavedModels/'+vars+'/ANN/ANN5.h5')

ANN7.save('/content/drive/MyDrive/Thesis/SavedModels/'+vars+'/ANN/ANN7.h5')

# **Random Forest**

## **Import Models**

If the models have already been develpoed, We can just import the saved models and move on to prediction. If not, then we have to train the models

In [ ]:
RF1= joblib.load('/content/drive/MyDrive/Thesis/SavedModels/'+vars+'/RF/RF1.h5')

RF3= joblib.load('/content/drive/MyDrive/Thesis/SavedModels/'+vars+'/RF/RF3.h5')

RF5= joblib.load('/content/drive/MyDrive/Thesis/SavedModels/'+vars+'/RF/RF5.h5')

RF7= joblib.load('/content/drive/MyDrive/Thesis/SavedModels/'+vars+'/RF/RF7.h5')

## **Model Build and train**

While building the models, number of esimators were set to 50, bootstrap was set to TRUE, oob_Score was set to TRUE, max_depth was set to 11, n_jobs set to -1 that inidicates the use of all the CPUs in the system, random state was set to 42. The model was then trained for the train dataset. 

In [ ]:
from sklearn.ensemble import RandomForestRegressor

RF1 = RandomForestRegressor(n_estimators=50, bootstrap=True, oob_score=True, max_depth=11, n_jobs=-1, random_state=42, verbose=2)
RF1.fit(trainX1,trainY1)

The same was repeated for the remaining lead times.

In [ ]:
RF3 = RandomForestRegressor(n_estimators=50, bootstrap=True, oob_score=True, max_depth=11, n_jobs=-1, random_state=42, verbose=2)
RF3.fit(trainX3,trainY3)

In [ ]:
RF5 = RandomForestRegressor(n_estimators=50, bootstrap=True, oob_score=True, max_depth=11, n_jobs=-1, random_state=42, verbose=2)
RF5.fit(trainX5,trainY5)

In [ ]:
RF7 = RandomForestRegressor(n_estimators=50, bootstrap=True, oob_score=True, max_depth=11, n_jobs=-1, random_state=42, verbose=2)
RF7.fit(trainX7,trainY7)

## **prediction**

It's time to make some predictions using our held out test dataset. we also use the train and validation dataset to calculate trianing and validation errors

In [ ]:
trainpredictRF1 = RF1.predict(trainX1)
valpredictRF1 = RF1.predict(valX1)
testpredictRF1 = RF1.predict(testX1)

trainpredictRF3 = RF3.predict(trainX3)
valpredictRF3 = RF3.predict(valX3)
testpredictRF3 = RF3.predict(testX3)

trainpredictRF5 = RF5.predict(trainX5)
valpredictRF5 = RF5.predict(valX5)
testpredictRF5 = RF5.predict(testX5)

trainpredictRF7 = RF7.predict(trainX7)
valpredictRF7 = RF7.predict(valX7)
testpredictRF7 = RF7.predict(testX7)

## **Evaluation**

Now we calculate RMSE values for train, test and validation datasets

In [ ]:
trainScoreRF1 = math.sqrt(mean_squared_error(trainY1, trainpredictRF1))
print('Train Score 1: %.2f RMSE' % (trainScoreRF1))
valScoreRF1 = math.sqrt(mean_squared_error(valY1, valpredictRF1))
print('val Score 1: %.2f RMSE' % (valScoreRF1))
testScoreRF1 = math.sqrt(mean_squared_error(testY1, testpredictRF1))
print('Test Score 1: %.2f RMSE' % (testScoreRF1))

trainScoreRF3 = math.sqrt(mean_squared_error(trainY3, trainpredictRF3))
print('Train Score 3: %.2f RMSE' % (trainScoreRF3))
valScoreRF3 = math.sqrt(mean_squared_error(valY3, valpredictRF3))
print('val Score 3: %.2f RMSE' % (valScoreRF3))
testScoreRF3 = math.sqrt(mean_squared_error(testY3, testpredictRF3))
print('Test Score 3: %.2f RMSE' % (testScoreRF3))

trainScoreRF5 = math.sqrt(mean_squared_error(trainY5, trainpredictRF5))
print('Train Score 5: %.2f RMSE' % (trainScoreRF5))
valScoreRF5 = math.sqrt(mean_squared_error(valY5, valpredictRF5))
print('val Score 5: %.2f RMSE' % (valScoreRF5))
testScoreRF5 = math.sqrt(mean_squared_error(testY5, testpredictRF5))
print('Test Score 5: %.2f RMSE' % (testScoreRF5))

trainScoreRF7 = math.sqrt(mean_squared_error(trainY7, trainpredictRF7))
print('Train Score 7: %.2f RMSE' % (trainScoreRF7))
valScoreRF7 = math.sqrt(mean_squared_error(valY7, valpredictRF7))
print('val Score 7: %.2f RMSE' % (valScoreRF7))
testScoreRF7 = math.sqrt(mean_squared_error(testY7, testpredictRF7))
print('Test Score 7: %.2f RMSE' % (testScoreRF7))

Train Score 1: 0.13 RMSE
val Score 1: 0.29 RMSE
Test Score 1: 0.37 RMSE
Train Score 3: 0.28 RMSE
val Score 3: 0.61 RMSE
Test Score 3: 0.75 RMSE
Train Score 5: 0.42 RMSE
val Score 5: 0.85 RMSE
Test Score 5: 1.02 RMSE
Train Score 7: 0.55 RMSE
val Score 7: 1.10 RMSE
Test Score 7: 1.28 RMSE


Now we calculate r2 values for train, test and validation datasets

In [ ]:
trainr2_RF1 =r2_score(trainY1, trainpredictRF1)
print('Train Score 1: %.2f RMSE' % (trainr2_RF1))
valr2_RF1 =r2_score(valY1, valpredictRF1)
print('val Score 1: %.2f RMSE' % (valr2_RF1))
testr2_RF1 =r2_score(testY1, testpredictRF1)
print('Test Score 1: %.2f RMSE' % (testr2_RF1))

trainr2_RF3 =r2_score(trainY3, trainpredictRF3)
print('Train Score 3: %.2f RMSE' % (trainr2_RF3))
valr2_RF3 =r2_score(valY3, valpredictRF3)
print('val Score 3: %.2f RMSE' % (valr2_RF3))
testr2_RF3 =r2_score(testY3, testpredictRF3)
print('Test Score 3: %.2f RMSE' % (testr2_RF3))

trainr2_RF5 =r2_score(trainY5, trainpredictRF5)
print('Train Score 5: %.2f RMSE' % (trainr2_RF5))
valr2_RF5 =r2_score(valY5, valpredictRF5)
print('val Score 5: %.2f RMSE' % (valr2_RF5))
testr2_RF5 =r2_score(testY5, testpredictRF5)
print('Test Score 5: %.2f RMSE' % (testr2_RF5))

trainr2_RF7 =r2_score(trainY7, trainpredictRF7)
print('Train Score 7: %.2f RMSE' % (trainr2_RF7))
valr2_RF7 =r2_score(valY7, valpredictRF7)
print('val Score 7: %.2f RMSE' % (valr2_RF7))
testr2_RF7 =r2_score(testY7, testpredictRF7)
print('Test Score 7: %.2f RMSE' % (testr2_RF7))

Train Score 1: 1.00 RMSE
val Score 1: 0.99 RMSE
Test Score 1: 0.99 RMSE
Train Score 3: 0.99 RMSE
val Score 3: 0.97 RMSE
Test Score 3: 0.95 RMSE
Train Score 5: 0.99 RMSE
val Score 5: 0.94 RMSE
Test Score 5: 0.91 RMSE
Train Score 7: 0.98 RMSE
val Score 7: 0.90 RMSE
Test Score 7: 0.86 RMSE


Now we edit the dataframe named df_rmse that contained the rmse values for the DNN model by adding the rmse scores for the RF model

In [ ]:
rmse_RFvals={'RF':[trainScoreRF1,valScoreRF1,testScoreRF1,trainScoreRF3,valScoreRF3,testScoreRF3,trainScoreRF5,valScoreRF5,testScoreRF5,trainScoreRF7,valScoreRF7,testScoreRF7]}
df_rmse_RF=pd.DataFrame(rmse_RFvals)
df_rmse=df_rmse.join(df_rmse_RF)



we do  the same for r2 values

In [ ]:
r2_RFvals={'RF':[trainr2_RF1,valr2_RF1,testr2_RF1,trainr2_RF3,valr2_RF3,testr2_RF3,trainr2_RF5,valr2_RF5,testr2_RF5,trainr2_RF7,valr2_RF7,testr2_RF7]}
df_r2_RF=pd.DataFrame(r2_RFvals)
df_r2=df_r2.join(df_r2_RF)

## **Save RF Models**

Time to save the models so they can be used later.

In [ ]:
import joblib

In [ ]:
joblib.dump(RF1,"/content/drive/MyDrive/Thesis/SavedModels/"+vars+"/RF/RF1.h5")

joblib.dump(RF3,"/content/drive/MyDrive/Thesis/SavedModels/"+vars+"/RF/RF3.h5")

joblib.dump(RF5,"/content/drive/MyDrive/Thesis/SavedModels/"+vars+"/RF/RF5.h5")

joblib.dump(RF7,"/content/drive/MyDrive/Thesis/SavedModels/"+vars+"/RF/RF7.h5")

['/content/drive/MyDrive/Thesis/SavedModels/TpSwvl/RF/RF7.h5']

# **CatboostRegressor**

## **Install CatBoostregressor**

First, We must install Catboost regresssor using pip

In [ ]:
pip install catboost

Now we load our required libraries

In [ ]:
from catboost import Pool,CatBoostRegressor

## **Load models**

Now we load the previoiusly saved models for making predictions. If we need to train the models, this section is to be ignored

In [ ]:
CB1= joblib.load('/content/drive/MyDrive/Thesis/SavedModels/'+vars+'/CB/CB1.h5')

CB3= joblib.load('/content/drive/MyDrive/Thesis/SavedModels/'+vars+'/CB/CB3.h5')

CB5= joblib.load('/content/drive/MyDrive/Thesis/SavedModels/'+vars+'/CB/CB5.h5')

CB7= joblib.load('/content/drive/MyDrive/Thesis/SavedModels/'+vars+'/CB/CB7.h5')

## **Build Models and Run**

to train the models, first we make a pool of data for the train and validation sets separately. then we build the model setting iterations to 8000, but keeping overfitting detector on. What this does is that it stops training of the models when the validation losses dont decrease for certain number of steps ( in our case 5 iterations) learning rate is set to 0.002, depth set to 10, loss function set to RMSE, and random seed set to 42. Finally the model was trained for the training dataset while setting the validation dataset for overfitting detection.

In [ ]:
CB1train = Pool(data=trainX1,
                label=trainY1)
CB1val = Pool(data=testX1,
              label=testY1)

CB1 = CatBoostRegressor(iterations=8000,
                        learning_rate=0.002,
                        depth=10,
                        l2_leaf_reg=20,
                        model_size_reg=None,
                        loss_function='RMSE',
                        od_wait=5,
                        od_type="Iter",
                        random_seed = 42)

CB1.fit(CB1train,
        use_best_model = True,
        eval_set = CB1val,
        )

The same was repeated for all lead times.

In [ ]:
CB3train = Pool(data=trainX3,
                label=trainY3)
CB3val = Pool(data=testX3,
              label=testY3)

CB3 = CatBoostRegressor(iterations=8000,
                        learning_rate=0.002,
                        depth=10,
                        l2_leaf_reg=20,
                        model_size_reg=None,
                        loss_function='RMSE',
                        od_wait=5,
                        od_type="Iter",
                        random_seed = 42)

CB3.fit(CB3train,
        use_best_model = True,
        eval_set = CB3val,
        )

In [ ]:
CB5train = Pool(data=trainX5,
                label=trainY5)
CB5val = Pool(data=testX5,
              label=testY5)

CB5 = CatBoostRegressor(iterations=8000,
                        learning_rate=0.002,
                        depth=10,
                        l2_leaf_reg=20,
                        model_size_reg=None,
                        loss_function='RMSE',
                        od_wait=5,
                        od_type="Iter",
                        random_seed = 42)

CB5.fit(CB1train,
        use_best_model = True,
        eval_set = CB5val,
        )

In [ ]:
CB7train = Pool(data=trainX7,
                label=trainY7)
CB7val = Pool(data=testX7,
              label=testY7)

CB7 = CatBoostRegressor(iterations=5000,
                        learning_rate=0.002,
                        depth=10,
                        l2_leaf_reg=20,
                        model_size_reg=None,
                        loss_function='RMSE',
                        od_wait=5,
                        od_type="Iter",
                        random_seed = 42)

CB7.fit(CB7train,
        use_best_model = True,
        eval_set = CB7val,
        )

## **Make predictions**

It's time to make some predictions using our held out test dataset. we also use the train and validation dataset to calculate trianing and validation errors

In [ ]:
trainpredictCB1 = CB1.predict(trainX1)
valpredictCB1 = CB1.predict(valX1)
testpredictCB1 = CB1.predict(testX1)

trainpredictCB3 = CB3.predict(trainX3)
valpredictCB3 = CB3.predict(valX3)
testpredictCB3 = CB3.predict(testX3)

trainpredictCB5 = CB5.predict(trainX5)
valpredictCB5 = CB5.predict(valX5)
testpredictCB5 = CB5.predict(testX5)

trainpredictCB7 = CB7.predict(trainX7)
valpredictCB7 = CB7.predict(valX7)
testpredictCB7 = CB7.predict(testX7)

## **Evaluate model**

Now we calculate RMSE values for train, test and validation datasets

In [ ]:
trainScoreCB1 = math.sqrt(mean_squared_error(trainY1, trainpredictCB1))
print('Train Score 1: %.2f RMSE' % (trainScoreCB1))
valScoreCB1 = math.sqrt(mean_squared_error(valY1, valpredictCB1))
print('val Score 1: %.2f RMSE' % (valScoreCB1))
testScoreCB1 = math.sqrt(mean_squared_error(testY1, testpredictCB1))
print('Test Score 1: %.2f RMSE' % (testScoreCB1))

trainScoreCB3 = math.sqrt(mean_squared_error(trainY3, trainpredictCB3))
print('Train Score 3: %.2f RMSE' % (trainScoreCB3))
valScoreCB3 = math.sqrt(mean_squared_error(valY3, valpredictCB3))
print('val Score 3: %.2f RMSE' % (valScoreCB3))
testScoreCB3 = math.sqrt(mean_squared_error(testY3, testpredictCB3))
print('Test Score 3: %.2f RMSE' % (testScoreCB3))

trainScoreCB5 = math.sqrt(mean_squared_error(trainY5, trainpredictCB5))
print('Train Score 5: %.2f RMSE' % (trainScoreCB5))
valScoreCB5 = math.sqrt(mean_squared_error(valY5, valpredictCB5))
print('val Score 5: %.2f RMSE' % (valScoreCB5))
testScoreCB5 = math.sqrt(mean_squared_error(testY5, testpredictCB5))
print('Test Score 5: %.2f RMSE' % (testScoreCB5))

trainScoreCB7 = math.sqrt(mean_squared_error(trainY7, trainpredictCB7))
print('Train Score 7: %.2f RMSE' % (trainScoreCB7))
valScoreCB7 = math.sqrt(mean_squared_error(valY7, valpredictCB7))
print('val Score 7: %.2f RMSE' % (valScoreCB7))
testScoreCB7 = math.sqrt(mean_squared_error(testY7, testpredictCB7))
print('Test Score 7: %.2f RMSE' % (testScoreCB7))

Now we calculate r2 values for train, test and validation datasets

In [ ]:
trainr2_CB1 =r2_score(trainY1, trainpredictCB1)
print('Train Score 1: %.2f RMSE' % (trainr2_CB1))
valr2_CB1 =r2_score(valY1, valpredictCB1)
print('val Score 1: %.2f RMSE' % (valr2_CB1))
testr2_CB1 =r2_score(testY1, testpredictCB1)
print('Test Score 1: %.2f RMSE' % (testr2_CB1))

trainr2_CB3 =r2_score(trainY3, trainpredictCB3)
print('Train Score 3: %.2f RMSE' % (trainr2_CB3))
valr2_CB3 =r2_score(valY3, valpredictCB3)
print('val Score 3: %.2f RMSE' % (valr2_CB3))
testr2_CB3 =r2_score(testY3, testpredictCB3)
print('Test Score 3: %.2f RMSE' % (testr2_CB3))

trainr2_CB5 =r2_score(trainY5, trainpredictCB5)
print('Train Score 5: %.2f RMSE' % (trainr2_CB5))
valr2_CB5 =r2_score(valY5, valpredictCB5)
print('val Score 5: %.2f RMSE' % (valr2_CB5))
testr2_CB5 =r2_score(testY5, testpredictCB5)
print('Test Score 5: %.2f RMSE' % (testr2_CB5))

trainr2_CB7 =r2_score(trainY7, trainpredictCB7)
print('Train Score 7: %.2f RMSE' % (trainr2_CB7))
valr2_CB7 =r2_score(valY7, valpredictCB7)
print('val Score 7: %.2f RMSE' % (valr2_CB7))
testr2_CB7 =r2_score(testY7, testpredictCB7)
print('Test Score 7: %.2f RMSE' % (testr2_CB7))

## **Edit Error df**

The dataframe containing rmse values are now updated for the new model also

In [ ]:
rmse_CBvals={'CB':[trainScoreCB1,valScoreCB1,testScoreCB1,trainScoreCB3,valScoreCB3,testScoreCB3,trainScoreCB5,valScoreCB5,testScoreCB5,trainScoreCB7,valScoreCB7,testScoreCB7]}
df_rmse_CB=pd.DataFrame(rmse_CBvals)
df_rmse=df_rmse.join(df_rmse_CB)



the same is done for the dataframe containing r2 values

In [ ]:
r2_CBvals={'CB':[trainr2_CB1,valr2_CB1,testr2_CB1,trainr2_CB3,valr2_CB3,testr2_CB3,trainr2_CB5,valr2_CB5,testr2_CB5,trainr2_CB7,valr2_CB7,testr2_CB7]}
df_r2_CB=pd.DataFrame(r2_CBvals)
df_r2=df_r2.join(df_r2_CB)

## **Show CB tree**

this code can help us visualise a tree layer 

In [ ]:
CB1.plot_tree(tree_idx=0,)

## **Save CB models**

Time to save the models for future use

In [ ]:
joblib.dump(CB1,"/content/drive/MyDrive/Thesis/SavedModels/"+vars+"/CB/CB1.h5")

joblib.dump(CB3,"/content/drive/MyDrive/Thesis/SavedModels/"+vars+"/CB/CB3.h5")

joblib.dump(CB5,"/content/drive/MyDrive/Thesis/SavedModels/"+vars+"/CB/CB5.h5")

joblib.dump(CB7,"/content/drive/MyDrive/Thesis/SavedModels/"+vars+"/CB/CB7.h5")

['/content/drive/MyDrive/Thesis/SavedModels/Tp/CB/CB7.h5']

# **LSTM**

If the models have already been develpoed, We can just import the saved models and move on to prediction. If not, then we have to train the models

## **Data Processing LSTM**

Processing data for the LSTM model is a little bit different than the other models.  First few steps, however, are similar

In [ ]:
dataframe

dfX and dfY are defined as the parent sets for input and target

In [ ]:
df = dataframe.set_index('Date')
dfX = df[list(df)[0:var]]
dfY=df[['WL']]

The data are now split into train, validation and test datasets

In [ ]:
train_size = int(len(dfX) * 0.5)
val_size = int(len(dfX) * 0.2)
test_size = len(dfX) - train_size - val_size

In [ ]:
Xtrain = dfX[:train_size]
print(Xtrain.shape)
Ytrain = dfY[:train_size]
print(Ytrain.shape)

Xval = dfX[train_size:-test_size]
print(Xval.shape)
Yval = dfY[train_size:-test_size]
print(Yval.shape)

Xtest = dfX[-test_size:]
print(Xtest.shape)
Ytest = dfY[-test_size:]
print(Ytest.shape)

Empty lists are created to be populated using formatted training data

In [ ]:
#Empty lists to be populated using formatted training data
trainX_LSTM1 = []
valX_LSTM1 = []
trainY_LSTM1 = []
testX_LSTM1 = []
valY_LSTM1 = []
testY_LSTM1 = []

trainX_LSTM3 = []
valX_LSTM3 = []
trainY_LSTM3 = []
testX_LSTM3 = []
valY_LSTM3 =[]
testY_LSTM3 = []

trainX_LSTM5 = []
trainY_LSTM5 = []
valX_LSTM5 = []
valY_LSTM5 = []
testX_LSTM5 = []
testY_LSTM5 = []

trainX_LSTM7 = []
trainY_LSTM7 = []
valX_LSTM7 = []
valY_LSTM7 = []
testX_LSTM7 = []
testY_LSTM7 = []

As required for LSTM networks, we require to reshape an input data into n_samples x timesteps x n_features.

n_future indicate Number of days we want to look into the future based on the past days.

n_past indicate Number of past days we want to use to predict the future.

In [ ]:
#As required for LSTM networks, we require to reshape an input data into n_samples x timesteps x n_features.
n_future1=1
n_future3=3
n_future5=5
n_future7=7   # Number of days we want to look into the future based on the past days.


n_past = 14  # Number of past days we want to use to predict the future.

Now we Reformat input data into a shape: (n_samples x timesteps x n_features) and convert them to numpy array

In [ ]:
#Reformat input data into a shape: (n_samples x timesteps x n_features)

for i1 in range(n_past, len(Xtrain) - n_future1 +1):
    trainX_LSTM1.append(Xtrain.iloc[i1 - n_past:i1, 0:Xtrain.shape[1]])
    trainY_LSTM1.append(Ytrain.iloc[i1 + n_future1 - 1])
for k1 in range(n_past, len(Xval) - n_future1 +1):
    valX_LSTM1.append(Xval.iloc[k1 - n_past:k1, 0:Xval.shape[1]])
    valY_LSTM1.append(Yval.iloc[k1 + n_future1 - 1])   
for j1 in range(n_past, len(Xtest) - n_future1 +1):
    testX_LSTM1.append(Xtest.iloc[j1 - n_past:j1, 0:Xtest.shape[1]])
    testY_LSTM1.append(Ytest.iloc[j1 + n_future1 - 1])

trainX_LSTM1, trainY_LSTM1 = np.array(trainX_LSTM1), np.array(trainY_LSTM1)
valX_LSTM1, valY_LSTM1 = np.array(valX_LSTM1), np.array(valY_LSTM1)
testX_LSTM1, testY_LSTM1 = np.array(testX_LSTM1), np.array(testY_LSTM1)


for i3 in range(n_past, len(Xtrain) - n_future3 +1):
    trainX_LSTM3.append(Xtrain.iloc[i3 - n_past:i3, 0:Xtrain.shape[1]])
    trainY_LSTM3.append(Ytrain.iloc[i3 + n_future3 - 1])
for k3 in range(n_past, len(Xval) - n_future3 +1):
    valX_LSTM3.append(Xval.iloc[k3 - n_past:k3, 0:Xval.shape[1]])
    valY_LSTM3.append(Yval.iloc[k3 + n_future3 - 1])   
for j3 in range(n_past, len(Xtest) - n_future3 +1):
    testX_LSTM3.append(Xtest.iloc[j3 - n_past:j3, 0:Xtest.shape[1]])
    testY_LSTM3.append(Ytest.iloc[j3 + n_future3 - 1])

trainX_LSTM3, trainY_LSTM3 = np.array(trainX_LSTM3), np.array(trainY_LSTM3)
valX_LSTM3, valY_LSTM3 = np.array(valX_LSTM3), np.array(valY_LSTM3)
testX_LSTM3, testY_LSTM3 = np.array(testX_LSTM3), np.array(testY_LSTM3)

for i5 in range(n_past, len(Xtrain) - n_future5 +1):
    trainX_LSTM5.append(Xtrain.iloc[i5 - n_past:i5, 0:Xtrain.shape[1]])
    trainY_LSTM5.append(Ytrain.iloc[i5 + n_future5 - 1])
for k5 in range(n_past, len(Xval) - n_future5 +1):
    valX_LSTM5.append(Xval.iloc[k5 - n_past:k5, 0:Xval.shape[1]])
    valY_LSTM5.append(Yval.iloc[k5 + n_future5 - 1])   
for j5 in range(n_past, len(Xtest) - n_future5 +1):
    testX_LSTM5.append(Xtest.iloc[j5 - n_past:j5, 0:Xtest.shape[1]])
    testY_LSTM5.append(Ytest.iloc[j5 + n_future5 - 1])

trainX_LSTM5, trainY_LSTM5 = np.array(trainX_LSTM5), np.array(trainY_LSTM5)
valX_LSTM5, valY_LSTM5 = np.array(valX_LSTM5), np.array(valY_LSTM5)
testX_LSTM5, testY_LSTM5 = np.array(testX_LSTM5), np.array(testY_LSTM5)

for i7 in range(n_past, len(Xtrain) - n_future7 +1):
    trainX_LSTM7.append(Xtrain.iloc[i7 - n_past:i7, 0:Xtrain.shape[1]])
    trainY_LSTM7.append(Ytrain.iloc[i7 + n_future7 - 1])
for k7 in range(n_past, len(Xval) - n_future7 +1):
    valX_LSTM7.append(Xval.iloc[k7 - n_past:k7, 0:Xval.shape[1]])
    valY_LSTM7.append(Yval.iloc[k7 + n_future7 - 1])   
for j7 in range(n_past, len(Xtest) - n_future7 +1):
    testX_LSTM7.append(Xtest.iloc[j7 - n_past:j7, 0:Xtest.shape[1]])
    testY_LSTM7.append(Ytest.iloc[j7 + n_future7 - 1])

trainX_LSTM7, trainY_LSTM7 = np.array(trainX_LSTM7), np.array(trainY_LSTM7)
valX_LSTM7, valY_LSTM7 = np.array(valX_LSTM7), np.array(valY_LSTM7)
testX_LSTM7, testY_LSTM7 = np.array(testX_LSTM7), np.array(testY_LSTM7)


We check the input shapes 

In [ ]:
print('trainX_LSTM1 shape == {}.'.format(trainX_LSTM1.shape))
print('trainY_LSTM1 shape == {}.'.format(trainY_LSTM1.shape))
print('testX_LSTM1 shape == {}.'.format(testX_LSTM1.shape))
print('testY_LSTM1 shape == {}.'.format(testY_LSTM1.shape))

print('trainX_LSTM3 shape == {}.'.format(trainX_LSTM3.shape))
print('trainY_LSTM3 shape == {}.'.format(trainY_LSTM3.shape))
print('testX_LSTM3 shape == {}.'.format(testX_LSTM3.shape))
print('testY_LSTM3 shape == {}.'.format(testY_LSTM3.shape))

print('trainX_LSTM5 shape == {}.'.format(trainX_LSTM5.shape))
print('trainY_LSTM5 shape == {}.'.format(trainY_LSTM5.shape))
print('testX_LSTM5 shape == {}.'.format(testX_LSTM5.shape))
print('testY_LSTM5 shape == {}.'.format(testY_LSTM5.shape))

print('trainX_LSTM7 shape == {}.'.format(trainX_LSTM7.shape))
print('trainY_LSTM7 shape == {}.'.format(trainY_LSTM7.shape))
print('testX_LSTM7 shape == {}.'.format(testX_LSTM7.shape))
print('testY_LSTM7 shape == {}.'.format(testY_LSTM7.shape))

trainX_LSTM1 shape == (2543, 14, 2).
trainY_LSTM1 shape == (2543, 1).
testX_LSTM1 shape == (1521, 14, 2).
testY_LSTM1 shape == (1521, 1).
trainX_LSTM3 shape == (2541, 14, 2).
trainY_LSTM3 shape == (2541, 1).
testX_LSTM3 shape == (1519, 14, 2).
testY_LSTM3 shape == (1519, 1).
trainX_LSTM5 shape == (2539, 14, 2).
trainY_LSTM5 shape == (2539, 1).
testX_LSTM5 shape == (1517, 14, 2).
testY_LSTM5 shape == (1517, 1).
trainX_LSTM7 shape == (2537, 14, 2).
trainY_LSTM7 shape == (2537, 1).
testX_LSTM7 shape == (1515, 14, 2).
testY_LSTM7 shape == (1515, 1).


## **Load Models**

We load earlier saved models from path. if not, then we proceeed to build models

In [ ]:
LSTM1=load_model('/content/drive/MyDrive/Thesis/SavedModels/'+vars+'/LSTM/LSTM1.h5')

LSTM3=load_model('/content/drive/MyDrive/Thesis/SavedModels/'+vars+'/LSTM/LSTM3.h5')

LSTM5=load_model('/content/drive/MyDrive/Thesis/SavedModels/'+vars+'/LSTM/LSTM5.h5')

LSTM7=load_model('/content/drive/MyDrive/Thesis/SavedModels/'+vars+'/LSTM/LSTM7.h5')

## **Building LSTM Models**

For building the DNN models, we define the model as sequential. then we add 3 LSTM layers with 64 neurons each, with activation function set to ReLU. the first layer contains the input variables and has the shape as the variables. The last dense layer was set with a single output. for the last LSTM model, Return sequences was set to False

finally, the models were compiled with loss function set to rmse and optimizer set to adam.

In [ ]:
# define the Autoencoder model

LSTM1 = Sequential()
LSTM1.add(LSTM(64, activation='relu', input_shape=(trainX_LSTM1.shape[1], trainX_LSTM1.shape[2]), return_sequences=True))
#LSTM1.add(Dropout(0.2))
#LSTM1.add(LSTM(64, activation='relu', return_sequences=True))
LSTM1.add(LSTM(64, activation='relu', return_sequences=True))
LSTM1.add(LSTM(64, activation='relu', return_sequences=False))
LSTM1.add(Dense(1))

LSTM1.compile(loss='mse', optimizer='Adam')
LSTM1.summary()

The same was repeated for all the time leads

In [ ]:
LSTM3 = Sequential()
LSTM3.add(LSTM(64, activation='relu', input_shape=(trainX_LSTM3.shape[1], trainX_LSTM3.shape[2]), return_sequences=True))
#LSTM3.add(Dropout(0.2))
#LSTM3.add(LSTM(64, activation='relu', return_sequences=True))
LSTM3.add(LSTM(64, activation='relu', return_sequences=True))
LSTM3.add(LSTM(64, activation='relu', return_sequences=False))
LSTM3.add(Dense(1))

LSTM3.compile(optimizer='adam', loss='mse')
LSTM3.summary()

In [ ]:
LSTM5 = Sequential()
LSTM5.add(LSTM(64, activation='relu', input_shape=(trainX_LSTM5.shape[1], trainX_LSTM5.shape[2]), return_sequences=True))
#LSTM5.add(Dropout(0.2))
#LSTM5.add(LSTM(32, activation='relu', return_sequences=True))
LSTM5.add(LSTM(64, activation='relu', return_sequences=True))
LSTM5.add(LSTM(64, activation='relu', return_sequences=False))
LSTM5.add(Dense(1))

LSTM5.compile(optimizer='adam', loss='mse')
LSTM5.summary()

In [ ]:
LSTM7 = Sequential()
LSTM7.add(LSTM(64, activation='relu', input_shape=(trainX_LSTM7.shape[1], trainX_LSTM7.shape[2]), return_sequences=True))
#LSTM7.add(Dropout(0.2))
#LSTM7.add(LSTM(32, activation='relu', return_sequences=True))
LSTM7.add(LSTM(64, activation='relu', return_sequences=True))
LSTM7.add(LSTM(64, activation='relu', return_sequences=False))
LSTM7.add(Dense(1))

LSTM7.compile(optimizer='adam', loss='mse')
LSTM7.summary()

## **Run**

First, we define Earlystopping as a callback with patience set to 5. this means that training will stop when validation error does not reduce for 5 epochs.

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

With a batch size set to 64, callback set to our defined callback, and validation data set to our defined validation data split, we train the models for upto 200 epochs. Training may stop long before that thanks to earlystopping calllback

In [ ]:
# fit the model
history1 =LSTM1.fit(trainX_LSTM1, trainY_LSTM1, epochs=200, batch_size=64, validation_data=[valX_LSTM1, valY_LSTM1], verbose=2, callbacks=[callback])

plt.plot(history1.history['loss'], label='Training loss')
plt.plot(history1.history['val_loss'], label='Validation loss')
plt.legend()

same was done for all the lead times

In [ ]:
history3 =LSTM3.fit(trainX_LSTM3, trainY_LSTM3, epochs=200, batch_size=64, validation_data=[valX_LSTM3, valY_LSTM3], verbose=2, callbacks=[callback])

plt.plot(history3.history['loss'], label='Training loss')
plt.plot(history3.history['val_loss'], label='Validation loss')
plt.legend()

In [ ]:
history5 =LSTM5.fit(trainX_LSTM5, trainY_LSTM5, epochs=200, batch_size=64, validation_data=[valX_LSTM5, valY_LSTM5], verbose=2, callbacks=[callback])

plt.plot(history5.history['loss'], label='Training loss')
plt.plot(history5.history['val_loss'], label='Validation loss')
plt.legend()

In [ ]:
history7 =LSTM7.fit(trainX_LSTM7, trainY_LSTM7, epochs=200, batch_size=64, validation_data=[valX_LSTM7, valY_LSTM7], verbose=2, callbacks=[callback])

plt.plot(history7.history['loss'], label='Training loss')
plt.plot(history7.history['val_loss'], label='Validation loss')
plt.legend()

## **prediction**

It's time to make some predictions using our held out test dataset. we also use the train and validation dataset to calculate trianing and validation errors

In [ ]:
trainpredictLSTM1 = LSTM1.predict(trainX_LSTM1)
valpredictLSTM1 = LSTM1.predict(valX_LSTM1)
testpredictLSTM1 = LSTM1.predict(testX_LSTM1)

trainpredictLSTM3 = LSTM3.predict(trainX_LSTM3)
valpredictLSTM3 = LSTM3.predict(valX_LSTM3)
testpredictLSTM3 = LSTM3.predict(testX_LSTM3)

trainpredictLSTM5 = LSTM5.predict(trainX_LSTM5)
valpredictLSTM5 = LSTM5.predict(valX_LSTM5)
testpredictLSTM5 = LSTM5.predict(testX_LSTM5)

trainpredictLSTM7 = LSTM7.predict(trainX_LSTM7)
valpredictLSTM7 = LSTM7.predict(valX_LSTM7)
testpredictLSTM7 = LSTM7.predict(testX_LSTM7)

## **Evaluation**

Now we calculate RMSE values for train, test and validation datasets

In [ ]:
trainScoreLSTM1 = math.sqrt(mean_squared_error(trainY_LSTM1, trainpredictLSTM1))
print('Train Score 1: %.2f RMSE' % (trainScoreLSTM1))
valScoreLSTM1 = math.sqrt(mean_squared_error(valY_LSTM1, valpredictLSTM1))
print('val Score 1: %.2f RMSE' % (valScoreLSTM1))
testScoreLSTM1 = math.sqrt(mean_squared_error(testY_LSTM1, testpredictLSTM1))
print('Test Score 1: %.2f RMSE' % (testScoreLSTM1))

trainScoreLSTM3 = math.sqrt(mean_squared_error(trainY_LSTM3, trainpredictLSTM3))
print('Train Score 3: %.2f RMSE' % (trainScoreLSTM3))
valScoreLSTM3 = math.sqrt(mean_squared_error(valY_LSTM3, valpredictLSTM3))
print('val Score 3: %.2f RMSE' % (valScoreLSTM3))
testScoreLSTM3 = math.sqrt(mean_squared_error(testY_LSTM3, testpredictLSTM3))
print('Test Score 3: %.2f RMSE' % (testScoreLSTM3))

trainScoreLSTM5 = math.sqrt(mean_squared_error(trainY_LSTM5, trainpredictLSTM5))
print('Train Score 5: %.2f RMSE' % (trainScoreLSTM5))
valScoreLSTM5 = math.sqrt(mean_squared_error(valY_LSTM5, valpredictLSTM5))
print('val Score 5: %.2f RMSE' % (valScoreLSTM5))
testScoreLSTM5 = math.sqrt(mean_squared_error(testY_LSTM5, testpredictLSTM5))
print('Test Score 5: %.2f RMSE' % (testScoreLSTM5))

trainScoreLSTM7 = math.sqrt(mean_squared_error(trainY_LSTM7, trainpredictLSTM7))
print('Train Score 7: %.2f RMSE' % (trainScoreLSTM7))
valScoreLSTM7 = math.sqrt(mean_squared_error(valY_LSTM7, valpredictLSTM7))
print('val Score 7: %.2f RMSE' % (valScoreLSTM7))
testScoreLSTM7 = math.sqrt(mean_squared_error(testY_LSTM7, testpredictLSTM7))
print('Test Score 7: %.2f RMSE' % (testScoreLSTM7))



Now we calculate r2 values for train, test and validation datasets

In [ ]:
trainr2_LSTM1 =r2_score(trainY_LSTM1, trainpredictLSTM1)
print('Train Score 1: %.2f RMSE' % (trainr2_LSTM1))
valr2_LSTM1 =r2_score(valY_LSTM1, valpredictLSTM1)
print('val Score 1: %.2f RMSE' % (valr2_LSTM1))
testr2_LSTM1 =r2_score(testY_LSTM1, testpredictLSTM1)
print('Test Score 1: %.2f RMSE' % (testr2_LSTM1))

trainr2_LSTM3 =r2_score(trainY_LSTM3, trainpredictLSTM3)
print('Train Score 3: %.2f RMSE' % (trainr2_LSTM3))
valr2_LSTM3 =r2_score(valY_LSTM3, valpredictLSTM3)
print('val Score 3: %.2f RMSE' % (valr2_LSTM3))
testr2_LSTM3 =r2_score(testY_LSTM3, testpredictLSTM3)
print('Test Score 3: %.2f RMSE' % (testr2_LSTM3))

trainr2_LSTM5 =r2_score(trainY_LSTM5, trainpredictLSTM5)
print('Train Score 5: %.2f RMSE' % (trainr2_LSTM5))
valr2_LSTM5 =r2_score(valY_LSTM5, valpredictLSTM5)
print('val Score 5: %.2f RMSE' % (valr2_LSTM5))
testr2_LSTM5 =r2_score(testY_LSTM5, testpredictLSTM5)
print('Test Score 5: %.2f RMSE' % (testr2_LSTM5))

trainr2_LSTM7 =r2_score(trainY_LSTM7, trainpredictLSTM7)
print('Train Score 7: %.2f RMSE' % (trainr2_LSTM7))
valr2_LSTM7 =r2_score(valY_LSTM7, valpredictLSTM7)
print('val Score 7: %.2f RMSE' % (valr2_LSTM7))
testr2_LSTM7 =r2_score(testY_LSTM7, testpredictLSTM7)
print('Test Score 7: %.2f RMSE' % (testr2_LSTM7))

We now update the rmse and r2 dataframe by including scores for our LSTM model

In [ ]:
rmse_LSTMvals={'LSTM':[trainScoreLSTM1,valScoreLSTM1,testScoreLSTM1,trainScoreLSTM3,valScoreLSTM3,testScoreLSTM3,trainScoreLSTM5,valScoreLSTM5,testScoreLSTM5,trainScoreLSTM7,valScoreLSTM7,testScoreLSTM7]}
df_rmse_LSTM=pd.DataFrame(rmse_LSTMvals)
df_rmse=df_rmse.join(df_rmse_LSTM)

r2_LSTMvals={'LSTM':[trainr2_LSTM1,valr2_LSTM1,testr2_LSTM1,trainr2_LSTM3,valr2_LSTM3,testr2_LSTM3,trainr2_LSTM5,valr2_LSTM5,testr2_LSTM5,trainr2_LSTM7,valr2_LSTM7,testr2_LSTM7]}
df_r2_LSTM=pd.DataFrame(r2_LSTMvals)
df_r2=df_r2.join(df_r2_LSTM)


## **Save LSTM models**

Time to save the new models for further use later

In [ ]:
LSTM1.save('/content/drive/MyDrive/Thesis/SavedModels/'+vars+'/LSTM/LSTM1.h5')

LSTM3.save('/content/drive/MyDrive/Thesis/SavedModels/'+vars+'/LSTM/LSTM3.h5')

LSTM5.save('/content/drive/MyDrive/Thesis/SavedModels/'+vars+'/LSTM/LSTM5.h5')

LSTM7.save('/content/drive/MyDrive/Thesis/SavedModels/'+vars+'/LSTM/LSTM7.h5')

# **Error dataframes**

In [ ]:
df_rmse

In [ ]:
df_r2

# **Monsoon Flood**

## **2017 Monsoon**

First we determine the water levels for monsoon season 2017. 

In [ ]:
WL2017Monsoon = testY1[-1310:-1173]
WL2017Monsoon

Now we predict the water levels for the 2017 monsoon flood period using our trained models for different lead times and calculate the RMSE values for each prediction.

Finally we create a dataframe with the RMSE values for 2017 monsoon flood

In [ ]:
predictANNY1_2017Monsoon=testpredictANN1[-1310:-1173].reshape(-1)
predictANNY3_2017Monsoon=testpredictANN3[-1310:-1173].reshape(-1)
predictANNY5_2017Monsoon=testpredictANN5[-1310:-1173].reshape(-1)
predictANNY7_2017Monsoon=testpredictANN7[-1310:-1173].reshape(-1)

rmseANN1day2017Monsoon= math.sqrt(mean_squared_error(predictANNY1_2017Monsoon,WL2017Monsoon))
print('RMSE ANN 1day 2017Monsoon = %.2f' %(rmseANN1day2017Monsoon))

rmseANN3day2017Monsoon= math.sqrt(mean_squared_error(predictANNY3_2017Monsoon,WL2017Monsoon))
print('RMSE ANN 3day 2017Monsoon = %.2f' %(rmseANN3day2017Monsoon))

rmseANN5day2017Monsoon= math.sqrt(mean_squared_error(predictANNY5_2017Monsoon,WL2017Monsoon))
print('RMSE ANN 5day 2017Monsoon = %.2f' %(rmseANN5day2017Monsoon))

rmseANN7day2017Monsoon= math.sqrt(mean_squared_error(predictANNY7_2017Monsoon,WL2017Monsoon))
print('RMSE ANN 7day 2017Monsoon = %.2f' %(rmseANN7day2017Monsoon))


predictRFY1_2017Monsoon=testpredictRF1[-1310:-1173].reshape(-1)
predictRFY3_2017Monsoon=testpredictRF3[-1310:-1173].reshape(-1)
predictRFY5_2017Monsoon=testpredictRF5[-1310:-1173].reshape(-1)
predictRFY7_2017Monsoon=testpredictRF7[-1310:-1173].reshape(-1)

rmseRF1day2017Monsoon= math.sqrt(mean_squared_error(predictRFY1_2017Monsoon,WL2017Monsoon))
print('RMSE RF 1day 2017Monsoon = %.2f' %(rmseRF1day2017Monsoon))

rmseRF3day2017Monsoon= math.sqrt(mean_squared_error(predictRFY3_2017Monsoon,WL2017Monsoon))
print('RMSE RF 3day 2017Monsoon = %.2f' %(rmseRF3day2017Monsoon))

rmseRF5day2017Monsoon= math.sqrt(mean_squared_error(predictRFY5_2017Monsoon,WL2017Monsoon))
print('RMSE RF 5day 2017Monsoon = %.2f' %(rmseRF5day2017Monsoon))

rmseRF7day2017Monsoon= math.sqrt(mean_squared_error(predictRFY7_2017Monsoon,WL2017Monsoon))
print('RMSE RF 7day 2017Monsoon = %.2f' %(rmseRF7day2017Monsoon))


predictCBY1_2017Monsoon=testpredictCB1[-1310:-1173].reshape(-1)
predictCBY3_2017Monsoon=testpredictCB3[-1310:-1173].reshape(-1)
predictCBY5_2017Monsoon=testpredictCB5[-1310:-1173].reshape(-1)
predictCBY7_2017Monsoon=testpredictCB7[-1310:-1173].reshape(-1)

rmseCB1day2017Monsoon= math.sqrt(mean_squared_error(predictCBY1_2017Monsoon,WL2017Monsoon))
print('RMSE CB 1day 2017Monsoon = %.2f' %(rmseCB1day2017Monsoon))

rmseCB3day2017Monsoon= math.sqrt(mean_squared_error(predictCBY3_2017Monsoon,WL2017Monsoon))
print('RMSE CB 3day 2017Monsoon = %.2f' %(rmseCB3day2017Monsoon))

rmseCB5day2017Monsoon= math.sqrt(mean_squared_error(predictCBY5_2017Monsoon,WL2017Monsoon))
print('RMSE CB 5day 2017Monsoon = %.2f' %(rmseCB5day2017Monsoon))

rmseCB7day2017Monsoon= math.sqrt(mean_squared_error(predictCBY7_2017Monsoon,WL2017Monsoon))
print('RMSE CB 7day 2017Monsoon = %.2f' %(rmseCB7day2017Monsoon))


predictLSTMY1_2017Monsoon=testpredictLSTM1[-1310:-1173].reshape(-1)
predictLSTMY3_2017Monsoon=testpredictLSTM3[-1310:-1173].reshape(-1)
predictLSTMY5_2017Monsoon=testpredictLSTM5[-1310:-1173].reshape(-1)
predictLSTMY7_2017Monsoon=testpredictLSTM7[-1310:-1173].reshape(-1)

rmseLSTM1day2017Monsoon= math.sqrt(mean_squared_error(predictLSTMY1_2017Monsoon,WL2017Monsoon))
print('RMSE LSTM 1day 2017Monsoon = %.2f' %(rmseLSTM1day2017Monsoon))

rmseLSTM3day2017Monsoon= math.sqrt(mean_squared_error(predictLSTMY3_2017Monsoon,WL2017Monsoon))
print('RMSE LSTM 3day 2017Monsoon = %.2f' %(rmseLSTM3day2017Monsoon))

rmseLSTM5day2017Monsoon= math.sqrt(mean_squared_error(predictLSTMY5_2017Monsoon,WL2017Monsoon))
print('RMSE LSTM 5day 2017Monsoon = %.2f' %(rmseLSTM5day2017Monsoon))

rmseLSTM7day2017Monsoon= math.sqrt(mean_squared_error(predictLSTMY7_2017Monsoon,WL2017Monsoon))
print('RMSE LSTM 7day 2017Monsoon = %.2f' %(rmseLSTM7day2017Monsoon))


rmse_2017Monsoon={'ANN':[rmseANN1day2017Monsoon,rmseANN3day2017Monsoon,rmseANN5day2017Monsoon,rmseANN7day2017Monsoon], 'RF':[rmseRF1day2017Monsoon,rmseRF3day2017Monsoon,rmseRF5day2017Monsoon,rmseRF7day2017Monsoon], 'CB':[rmseCB1day2017Monsoon,rmseCB3day2017Monsoon,rmseCB5day2017Monsoon,rmseCB7day2017Monsoon], 'LSTM': [rmseLSTM1day2017Monsoon,rmseLSTM3day2017Monsoon,rmseLSTM5day2017Monsoon,rmseLSTM7day2017Monsoon]}
df_rmse_2017Monsoon=pd.DataFrame(rmse_2017Monsoon)

In [ ]:
df_rmse_2017Monsoon

## **2018 Monsson**

NOw we determine the water levels for monsoon season 2018.

In [ ]:
WL2018Monsoon = testY1[-1310+365:-1173+365]
WL2018Monsoon

Now we predict the water levels for the 2018 monsoon flood period using our trained models for different lead times and calculate the RMSE values for each prediction.

Finally we create a dataframe with the RMSE values for 2018 monsoon flood

In [ ]:
predictANNY1_2018Monsoon=testpredictANN1[-1310+365:-1173+365].reshape(-1)
predictANNY3_2018Monsoon=testpredictANN3[-1310+365:-1173+365].reshape(-1)
predictANNY5_2018Monsoon=testpredictANN5[-1310+365:-1173+365].reshape(-1)
predictANNY7_2018Monsoon=testpredictANN7[-1310+365:-1173+365].reshape(-1)

rmseANN1day2018Monsoon= math.sqrt(mean_squared_error(predictANNY1_2018Monsoon,WL2018Monsoon))
print('RMSE ANN 1day 2018Monsoon = %.2f' %(rmseANN1day2018Monsoon))

rmseANN3day2018Monsoon= math.sqrt(mean_squared_error(predictANNY3_2018Monsoon,WL2018Monsoon))
print('RMSE ANN 3day 2018Monsoon = %.2f' %(rmseANN3day2018Monsoon))

rmseANN5day2018Monsoon= math.sqrt(mean_squared_error(predictANNY5_2018Monsoon,WL2018Monsoon))
print('RMSE ANN 5day 2018Monsoon = %.2f' %(rmseANN5day2018Monsoon))

rmseANN7day2018Monsoon= math.sqrt(mean_squared_error(predictANNY7_2018Monsoon,WL2018Monsoon))
print('RMSE ANN 7day 2018Monsoon = %.2f' %(rmseANN7day2018Monsoon))


predictRFY1_2018Monsoon=testpredictRF1[-1310+365:-1173+365].reshape(-1)
predictRFY3_2018Monsoon=testpredictRF3[-1310+365:-1173+365].reshape(-1)
predictRFY5_2018Monsoon=testpredictRF5[-1310+365:-1173+365].reshape(-1)
predictRFY7_2018Monsoon=testpredictRF7[-1310+365:-1173+365].reshape(-1)

rmseRF1day2018Monsoon= math.sqrt(mean_squared_error(predictRFY1_2018Monsoon,WL2018Monsoon))
print('RMSE RF 1day 2018Monsoon = %.2f' %(rmseRF1day2018Monsoon))

rmseRF3day2018Monsoon= math.sqrt(mean_squared_error(predictRFY3_2018Monsoon,WL2018Monsoon))
print('RMSE RF 3day 2018Monsoon = %.2f' %(rmseRF3day2018Monsoon))

rmseRF5day2018Monsoon= math.sqrt(mean_squared_error(predictRFY5_2018Monsoon,WL2018Monsoon))
print('RMSE RF 5day 2018Monsoon = %.2f' %(rmseRF5day2018Monsoon))

rmseRF7day2018Monsoon= math.sqrt(mean_squared_error(predictRFY7_2018Monsoon,WL2018Monsoon))
print('RMSE RF 7day 2018Monsoon = %.2f' %(rmseRF7day2018Monsoon))


predictCBY1_2018Monsoon=testpredictCB1[-1310+365:-1173+365].reshape(-1)
predictCBY3_2018Monsoon=testpredictCB3[-1310+365:-1173+365].reshape(-1)
predictCBY5_2018Monsoon=testpredictCB5[-1310+365:-1173+365].reshape(-1)
predictCBY7_2018Monsoon=testpredictCB7[-1310+365:-1173+365].reshape(-1)

rmseCB1day2018Monsoon= math.sqrt(mean_squared_error(predictCBY1_2018Monsoon,WL2018Monsoon))
print('RMSE CB 1day 2018Monsoon = %.2f' %(rmseCB1day2018Monsoon))

rmseCB3day2018Monsoon= math.sqrt(mean_squared_error(predictCBY3_2018Monsoon,WL2018Monsoon))
print('RMSE CB 3day 2018Monsoon = %.2f' %(rmseCB3day2018Monsoon))

rmseCB5day2018Monsoon= math.sqrt(mean_squared_error(predictCBY5_2018Monsoon,WL2018Monsoon))
print('RMSE CB 5day 2018Monsoon = %.2f' %(rmseCB5day2018Monsoon))

rmseCB7day2018Monsoon= math.sqrt(mean_squared_error(predictCBY7_2018Monsoon,WL2018Monsoon))
print('RMSE CB 7day 2018Monsoon = %.2f' %(rmseCB7day2018Monsoon))


predictLSTMY1_2018Monsoon=testpredictLSTM1[-1310+365:-1173+365].reshape(-1)
predictLSTMY3_2018Monsoon=testpredictLSTM3[-1310+365:-1173+365].reshape(-1)
predictLSTMY5_2018Monsoon=testpredictLSTM5[-1310+365:-1173+365].reshape(-1)
predictLSTMY7_2018Monsoon=testpredictLSTM7[-1310+365:-1173+365].reshape(-1)

rmseLSTM1day2018Monsoon= math.sqrt(mean_squared_error(predictLSTMY1_2018Monsoon,WL2018Monsoon))
print('RMSE LSTM 1day 2018Monsoon = %.2f' %(rmseLSTM1day2018Monsoon))

rmseLSTM3day2018Monsoon= math.sqrt(mean_squared_error(predictLSTMY3_2018Monsoon,WL2018Monsoon))
print('RMSE LSTM 3day 2018Monsoon = %.2f' %(rmseLSTM3day2018Monsoon))

rmseLSTM5day2018Monsoon= math.sqrt(mean_squared_error(predictLSTMY5_2018Monsoon,WL2018Monsoon))
print('RMSE LSTM 5day 2018Monsoon = %.2f' %(rmseLSTM5day2018Monsoon))

rmseLSTM7day2018Monsoon= math.sqrt(mean_squared_error(predictLSTMY7_2018Monsoon,WL2018Monsoon))
print('RMSE LSTM 7day 2018Monsoon = %.2f' %(rmseLSTM7day2018Monsoon))


rmse_2018Monsoon={'ANN':[rmseANN1day2018Monsoon,rmseANN3day2018Monsoon,rmseANN5day2018Monsoon,rmseANN7day2018Monsoon], 'RF':[rmseRF1day2018Monsoon,rmseRF3day2018Monsoon,rmseRF5day2018Monsoon,rmseRF7day2018Monsoon], 'CB':[rmseCB1day2018Monsoon,rmseCB3day2018Monsoon,rmseCB5day2018Monsoon,rmseCB7day2018Monsoon], 'LSTM': [rmseLSTM1day2018Monsoon,rmseLSTM3day2018Monsoon,rmseLSTM5day2018Monsoon,rmseLSTM7day2018Monsoon]}
df_rmse_2018Monsoon=pd.DataFrame(rmse_2018Monsoon)

In [ ]:
df_rmse_2018Monsoon

## **2019 Monsoon**

Now we determine the water levels for monsoon season 2019.

In [ ]:
WL2019Monsoon = testY1[-1310+365+365:-1173+365+365]
WL2019Monsoon

Now we predict the water levels for the 2019 monsoon flood period using our trained models for different lead times and calculate the RMSE values for each prediction.

Finally we create a dataframe with the RMSE values for 2019 monsoon flood

In [ ]:
predictANNY1_2019Monsoon=testpredictANN1[-1310+365+365:-1173+365+365].reshape(-1)
predictANNY3_2019Monsoon=testpredictANN3[-1310+365+365:-1173+365+365].reshape(-1)
predictANNY5_2019Monsoon=testpredictANN5[-1310+365+365:-1173+365+365].reshape(-1)
predictANNY7_2019Monsoon=testpredictANN7[-1310+365+365:-1173+365+365].reshape(-1)

rmseANN1day2019Monsoon= math.sqrt(mean_squared_error(predictANNY1_2019Monsoon,WL2019Monsoon))
print('RMSE ANN 1day 2019Monsoon = %.2f' %(rmseANN1day2019Monsoon))

rmseANN3day2019Monsoon= math.sqrt(mean_squared_error(predictANNY3_2019Monsoon,WL2019Monsoon))
print('RMSE ANN 3day 2019Monsoon = %.2f' %(rmseANN3day2019Monsoon))

rmseANN5day2019Monsoon= math.sqrt(mean_squared_error(predictANNY5_2019Monsoon,WL2019Monsoon))
print('RMSE ANN 5day 2019Monsoon = %.2f' %(rmseANN5day2019Monsoon))

rmseANN7day2019Monsoon= math.sqrt(mean_squared_error(predictANNY7_2019Monsoon,WL2019Monsoon))
print('RMSE ANN 7day 2019Monsoon = %.2f' %(rmseANN7day2019Monsoon))


predictRFY1_2019Monsoon=testpredictRF1[-1310+365+365:-1173+365+365].reshape(-1)
predictRFY3_2019Monsoon=testpredictRF3[-1310+365+365:-1173+365+365].reshape(-1)
predictRFY5_2019Monsoon=testpredictRF5[-1310+365+365:-1173+365+365].reshape(-1)
predictRFY7_2019Monsoon=testpredictRF7[-1310+365+365:-1173+365+365].reshape(-1)

rmseRF1day2019Monsoon= math.sqrt(mean_squared_error(predictRFY1_2019Monsoon,WL2019Monsoon))
print('RMSE RF 1day 2019Monsoon = %.2f' %(rmseRF1day2019Monsoon))

rmseRF3day2019Monsoon= math.sqrt(mean_squared_error(predictRFY3_2019Monsoon,WL2019Monsoon))
print('RMSE RF 3day 2019Monsoon = %.2f' %(rmseRF3day2019Monsoon))

rmseRF5day2019Monsoon= math.sqrt(mean_squared_error(predictRFY5_2019Monsoon,WL2019Monsoon))
print('RMSE RF 5day 2019Monsoon = %.2f' %(rmseRF5day2019Monsoon))

rmseRF7day2019Monsoon= math.sqrt(mean_squared_error(predictRFY7_2019Monsoon,WL2019Monsoon))
print('RMSE RF 7day 2019Monsoon = %.2f' %(rmseRF7day2019Monsoon))


predictCBY1_2019Monsoon=testpredictCB1[-1310+365+365:-1173+365+365].reshape(-1)
predictCBY3_2019Monsoon=testpredictCB3[-1310+365+365:-1173+365+365].reshape(-1)
predictCBY5_2019Monsoon=testpredictCB5[-1310+365+365:-1173+365+365].reshape(-1)
predictCBY7_2019Monsoon=testpredictCB7[-1310+365+365:-1173+365+365].reshape(-1)

rmseCB1day2019Monsoon= math.sqrt(mean_squared_error(predictCBY1_2019Monsoon,WL2019Monsoon))
print('RMSE CB 1day 2019Monsoon = %.2f' %(rmseCB1day2019Monsoon))

rmseCB3day2019Monsoon= math.sqrt(mean_squared_error(predictCBY3_2019Monsoon,WL2019Monsoon))
print('RMSE CB 3day 2019Monsoon = %.2f' %(rmseCB3day2019Monsoon))

rmseCB5day2019Monsoon= math.sqrt(mean_squared_error(predictCBY5_2019Monsoon,WL2019Monsoon))
print('RMSE CB 5day 2019Monsoon = %.2f' %(rmseCB5day2019Monsoon))

rmseCB7day2019Monsoon= math.sqrt(mean_squared_error(predictCBY7_2019Monsoon,WL2019Monsoon))
print('RMSE CB 7day 2019Monsoon = %.2f' %(rmseCB7day2019Monsoon))


predictLSTMY1_2019Monsoon=testpredictLSTM1[-1310+365+365:-1173+365+365].reshape(-1)
predictLSTMY3_2019Monsoon=testpredictLSTM3[-1310+365+365:-1173+365+365].reshape(-1)
predictLSTMY5_2019Monsoon=testpredictLSTM5[-1310+365+365:-1173+365+365].reshape(-1)
predictLSTMY7_2019Monsoon=testpredictLSTM7[-1310+365+365:-1173+365+365].reshape(-1)

rmseLSTM1day2019Monsoon= math.sqrt(mean_squared_error(predictLSTMY1_2019Monsoon,WL2019Monsoon))
print('RMSE LSTM 1day 2019Monsoon = %.2f' %(rmseLSTM1day2019Monsoon))

rmseLSTM3day2019Monsoon= math.sqrt(mean_squared_error(predictLSTMY3_2019Monsoon,WL2019Monsoon))
print('RMSE LSTM 3day 2019Monsoon = %.2f' %(rmseLSTM3day2019Monsoon))

rmseLSTM5day2019Monsoon= math.sqrt(mean_squared_error(predictLSTMY5_2019Monsoon,WL2019Monsoon))
print('RMSE LSTM 5day 2019Monsoon = %.2f' %(rmseLSTM5day2019Monsoon))

rmseLSTM7day2019Monsoon= math.sqrt(mean_squared_error(predictLSTMY7_2019Monsoon,WL2019Monsoon))
print('RMSE LSTM 7day 2019Monsoon = %.2f' %(rmseLSTM7day2019Monsoon))


rmse_2019Monsoon={'ANN':[rmseANN1day2019Monsoon,rmseANN3day2019Monsoon,rmseANN5day2019Monsoon,rmseANN7day2019Monsoon], 'RF':[rmseRF1day2019Monsoon,rmseRF3day2019Monsoon,rmseRF5day2019Monsoon,rmseRF7day2019Monsoon], 'CB':[rmseCB1day2019Monsoon,rmseCB3day2019Monsoon,rmseCB5day2019Monsoon,rmseCB7day2019Monsoon], 'LSTM': [rmseLSTM1day2019Monsoon,rmseLSTM3day2019Monsoon,rmseLSTM5day2019Monsoon,rmseLSTM7day2019Monsoon]}
df_rmse_2019Monsoon=pd.DataFrame(rmse_2019Monsoon)

In [ ]:
df_rmse_2019Monsoon

## **2020 Monsson**

we determine the water levels for monsoon season 2020.

In [ ]:
WL2020Monsoon = testY1[-1310+365+365+366:-1173+365+365+366]
WL2020Monsoon

Now we predict the water levels for the 2020 monsoon flood period using our trained models for different lead times and calculate the RMSE values for each prediction.

Finally we create a dataframe with the RMSE values for 2020 monsoon flood

In [ ]:
predictANNY1_2020Monsoon=testpredictANN1[-1310+365+365+366:-1173+365+365+366].reshape(-1)
predictANNY3_2020Monsoon=testpredictANN3[-1310+365+365+366:-1173+365+365+366].reshape(-1)
predictANNY5_2020Monsoon=testpredictANN5[-1310+365+365+366:-1173+365+365+366].reshape(-1)
predictANNY7_2020Monsoon=testpredictANN7[-1310+365+365+366:-1173+365+365+366].reshape(-1)

rmseANN1day2020Monsoon= math.sqrt(mean_squared_error(predictANNY1_2020Monsoon,WL2020Monsoon))
print('RMSE ANN 1day 2020Monsoon = %.2f' %(rmseANN1day2020Monsoon))

rmseANN3day2020Monsoon= math.sqrt(mean_squared_error(predictANNY3_2020Monsoon,WL2020Monsoon))
print('RMSE ANN 3day 2020Monsoon = %.2f' %(rmseANN3day2020Monsoon))

rmseANN5day2020Monsoon= math.sqrt(mean_squared_error(predictANNY5_2020Monsoon,WL2020Monsoon))
print('RMSE ANN 5day 2020Monsoon = %.2f' %(rmseANN5day2020Monsoon))

rmseANN7day2020Monsoon= math.sqrt(mean_squared_error(predictANNY7_2020Monsoon,WL2020Monsoon))
print('RMSE ANN 7day 2020Monsoon = %.2f' %(rmseANN7day2020Monsoon))


predictRFY1_2020Monsoon=testpredictRF1[-1310+365+365+366:-1173+365+365+366].reshape(-1)
predictRFY3_2020Monsoon=testpredictRF3[-1310+365+365+366:-1173+365+365+366].reshape(-1)
predictRFY5_2020Monsoon=testpredictRF5[-1310+365+365+366:-1173+365+365+366].reshape(-1)
predictRFY7_2020Monsoon=testpredictRF7[-1310+365+365+366:-1173+365+365+366].reshape(-1)

rmseRF1day2020Monsoon= math.sqrt(mean_squared_error(predictRFY1_2020Monsoon,WL2020Monsoon))
print('RMSE RF 1day 2020Monsoon = %.2f' %(rmseRF1day2020Monsoon))

rmseRF3day2020Monsoon= math.sqrt(mean_squared_error(predictRFY3_2020Monsoon,WL2020Monsoon))
print('RMSE RF 3day 2020Monsoon = %.2f' %(rmseRF3day2020Monsoon))

rmseRF5day2020Monsoon= math.sqrt(mean_squared_error(predictRFY5_2020Monsoon,WL2020Monsoon))
print('RMSE RF 5day 2020Monsoon = %.2f' %(rmseRF5day2020Monsoon))

rmseRF7day2020Monsoon= math.sqrt(mean_squared_error(predictRFY7_2020Monsoon,WL2020Monsoon))
print('RMSE RF 7day 2020Monsoon = %.2f' %(rmseRF7day2020Monsoon))


predictCBY1_2020Monsoon=testpredictCB1[-1310+365+365+366:-1173+365+365+366].reshape(-1)
predictCBY3_2020Monsoon=testpredictCB3[-1310+365+365+366:-1173+365+365+366].reshape(-1)
predictCBY5_2020Monsoon=testpredictCB5[-1310+365+365+366:-1173+365+365+366].reshape(-1)
predictCBY7_2020Monsoon=testpredictCB7[-1310+365+365+366:-1173+365+365+366].reshape(-1)

rmseCB1day2020Monsoon= math.sqrt(mean_squared_error(predictCBY1_2020Monsoon,WL2020Monsoon))
print('RMSE CB 1day 2020Monsoon = %.2f' %(rmseCB1day2020Monsoon))

rmseCB3day2020Monsoon= math.sqrt(mean_squared_error(predictCBY3_2020Monsoon,WL2020Monsoon))
print('RMSE CB 3day 2020Monsoon = %.2f' %(rmseCB3day2020Monsoon))

rmseCB5day2020Monsoon= math.sqrt(mean_squared_error(predictCBY5_2020Monsoon,WL2020Monsoon))
print('RMSE CB 5day 2020Monsoon = %.2f' %(rmseCB5day2020Monsoon))

rmseCB7day2020Monsoon= math.sqrt(mean_squared_error(predictCBY7_2020Monsoon,WL2020Monsoon))
print('RMSE CB 7day 2020Monsoon = %.2f' %(rmseCB7day2020Monsoon))


predictLSTMY1_2020Monsoon=testpredictLSTM1[-1310+365+365+366:-1173+365+365+366].reshape(-1)
predictLSTMY3_2020Monsoon=testpredictLSTM3[-1310+365+365+366:-1173+365+365+366].reshape(-1)
predictLSTMY5_2020Monsoon=testpredictLSTM5[-1310+365+365+366:-1173+365+365+366].reshape(-1)
predictLSTMY7_2020Monsoon=testpredictLSTM7[-1310+365+365+366:-1173+365+365+366].reshape(-1)

rmseLSTM1day2020Monsoon= math.sqrt(mean_squared_error(predictLSTMY1_2020Monsoon,WL2020Monsoon))
print('RMSE LSTM 1day 2020Monsoon = %.2f' %(rmseLSTM1day2020Monsoon))

rmseLSTM3day2020Monsoon= math.sqrt(mean_squared_error(predictLSTMY3_2020Monsoon,WL2020Monsoon))
print('RMSE LSTM 3day 2020Monsoon = %.2f' %(rmseLSTM3day2020Monsoon))

rmseLSTM5day2020Monsoon= math.sqrt(mean_squared_error(predictLSTMY5_2020Monsoon,WL2020Monsoon))
print('RMSE LSTM 5day 2020Monsoon = %.2f' %(rmseLSTM5day2020Monsoon))

rmseLSTM7day2020Monsoon= math.sqrt(mean_squared_error(predictLSTMY7_2020Monsoon,WL2020Monsoon))
print('RMSE LSTM 7day 2020Monsoon = %.2f' %(rmseLSTM7day2020Monsoon))


rmse_2020Monsoon={'ANN':[rmseANN1day2020Monsoon,rmseANN3day2020Monsoon,rmseANN5day2020Monsoon,rmseANN7day2020Monsoon], 'RF':[rmseRF1day2020Monsoon,rmseRF3day2020Monsoon,rmseRF5day2020Monsoon,rmseRF7day2020Monsoon], 'CB':[rmseCB1day2020Monsoon,rmseCB3day2020Monsoon,rmseCB5day2020Monsoon,rmseCB7day2020Monsoon], 'LSTM': [rmseLSTM1day2020Monsoon,rmseLSTM3day2020Monsoon,rmseLSTM5day2020Monsoon,rmseLSTM7day2020Monsoon]}
df_rmse_2020Monsoon=pd.DataFrame(rmse_2020Monsoon)

In [ ]:
df_rmse_2020Monsoon

## **Monsoon Flood df**

Now we create a dataframe and merge all the RMSE values for the monsoon flood events.

In [ ]:
rmse_Monsoon = pd.concat([df_rmse_2017Monsoon, df_rmse_2018Monsoon, df_rmse_2019Monsoon, df_rmse_2020Monsoon ],axis=1)
rmse_Monsoon

# **Flash Floods**

## **Flash Flood 2017**

we determine the water levels for Flash flood season 2017.

In [ ]:
WL2017Flash = testY1[-1310-78:-1310]
WL2017Flash

Now we predict the water levels for the 2017 Flash flood period using our trained models for different lead times and calculate the RMSE values for each prediction.

Finally we create a dataframe with the RMSE values for 2017 Flash flood

In [ ]:
predictANNY1_2017Flash=testpredictANN1[-1310-78:-1310].reshape(-1)
predictANNY3_2017Flash=testpredictANN3[-1310-78:-1310].reshape(-1)
predictANNY5_2017Flash=testpredictANN5[-1310-78:-1310].reshape(-1)
predictANNY7_2017Flash=testpredictANN7[-1310-78:-1310].reshape(-1)

rmseANN1day2017Flash= math.sqrt(mean_squared_error(predictANNY1_2017Flash,WL2017Flash))
print('RMSE ANN 1day 2017Flash = %.2f' %(rmseANN1day2017Flash))

rmseANN3day2017Flash= math.sqrt(mean_squared_error(predictANNY3_2017Flash,WL2017Flash))
print('RMSE ANN 3day 2017Flash = %.2f' %(rmseANN3day2017Flash))

rmseANN5day2017Flash= math.sqrt(mean_squared_error(predictANNY5_2017Flash,WL2017Flash))
print('RMSE ANN 5day 2017Flash = %.2f' %(rmseANN5day2017Flash))

rmseANN7day2017Flash= math.sqrt(mean_squared_error(predictANNY7_2017Flash,WL2017Flash))
print('RMSE ANN 7day 2017Flash = %.2f' %(rmseANN7day2017Flash))


predictRFY1_2017Flash=testpredictRF1[-1310-78:-1310].reshape(-1)
predictRFY3_2017Flash=testpredictRF3[-1310-78:-1310].reshape(-1)
predictRFY5_2017Flash=testpredictRF5[-1310-78:-1310].reshape(-1)
predictRFY7_2017Flash=testpredictRF7[-1310-78:-1310].reshape(-1)

rmseRF1day2017Flash= math.sqrt(mean_squared_error(predictRFY1_2017Flash,WL2017Flash))
print('RMSE RF 1day 2017Flash = %.2f' %(rmseRF1day2017Flash))

rmseRF3day2017Flash= math.sqrt(mean_squared_error(predictRFY3_2017Flash,WL2017Flash))
print('RMSE RF 3day 2017Flash = %.2f' %(rmseRF3day2017Flash))

rmseRF5day2017Flash= math.sqrt(mean_squared_error(predictRFY5_2017Flash,WL2017Flash))
print('RMSE RF 5day 2017Flash = %.2f' %(rmseRF5day2017Flash))

rmseRF7day2017Flash= math.sqrt(mean_squared_error(predictRFY7_2017Flash,WL2017Flash))
print('RMSE RF 7day 2017Flash = %.2f' %(rmseRF7day2017Flash))


predictCBY1_2017Flash=testpredictCB1[-1310-78:-1310].reshape(-1)
predictCBY3_2017Flash=testpredictCB3[-1310-78:-1310].reshape(-1)
predictCBY5_2017Flash=testpredictCB5[-1310-78:-1310].reshape(-1)
predictCBY7_2017Flash=testpredictCB7[-1310-78:-1310].reshape(-1)

rmseCB1day2017Flash= math.sqrt(mean_squared_error(predictCBY1_2017Flash,WL2017Flash))
print('RMSE CB 1day 2017Flash = %.2f' %(rmseCB1day2017Flash))

rmseCB3day2017Flash= math.sqrt(mean_squared_error(predictCBY3_2017Flash,WL2017Flash))
print('RMSE CB 3day 2017Flash = %.2f' %(rmseCB3day2017Flash))

rmseCB5day2017Flash= math.sqrt(mean_squared_error(predictCBY5_2017Flash,WL2017Flash))
print('RMSE CB 5day 2017Flash = %.2f' %(rmseCB5day2017Flash))

rmseCB7day2017Flash= math.sqrt(mean_squared_error(predictCBY7_2017Flash,WL2017Flash))
print('RMSE CB 7day 2017Flash = %.2f' %(rmseCB7day2017Flash))


predictLSTMY1_2017Flash=testpredictLSTM1[-1310-78:-1310].reshape(-1)
predictLSTMY3_2017Flash=testpredictLSTM3[-1310-78:-1310].reshape(-1)
predictLSTMY5_2017Flash=testpredictLSTM5[-1310-78:-1310].reshape(-1)
predictLSTMY7_2017Flash=testpredictLSTM7[-1310-78:-1310].reshape(-1)

rmseLSTM1day2017Flash= math.sqrt(mean_squared_error(predictLSTMY1_2017Flash,WL2017Flash))
print('RMSE LSTM 1day 2017Flash = %.2f' %(rmseLSTM1day2017Flash))

rmseLSTM3day2017Flash= math.sqrt(mean_squared_error(predictLSTMY3_2017Flash,WL2017Flash))
print('RMSE LSTM 3day 2017Flash = %.2f' %(rmseLSTM3day2017Flash))

rmseLSTM5day2017Flash= math.sqrt(mean_squared_error(predictLSTMY5_2017Flash,WL2017Flash))
print('RMSE LSTM 5day 2017Flash = %.2f' %(rmseLSTM5day2017Flash))

rmseLSTM7day2017Flash= math.sqrt(mean_squared_error(predictLSTMY7_2017Flash,WL2017Flash))
print('RMSE LSTM 7day 2017Flash = %.2f' %(rmseLSTM7day2017Flash))


rmse_2017Flash={'ANN':[rmseANN1day2017Flash,rmseANN3day2017Flash,rmseANN5day2017Flash,rmseANN7day2017Flash], 'RF':[rmseRF1day2017Flash,rmseRF3day2017Flash,rmseRF5day2017Flash,rmseRF7day2017Flash], 'CB':[rmseCB1day2017Flash,rmseCB3day2017Flash,rmseCB5day2017Flash,rmseCB7day2017Flash], 'LSTM': [rmseLSTM1day2017Flash,rmseLSTM3day2017Flash,rmseLSTM5day2017Flash,rmseLSTM7day2017Flash]}
df_rmse_2017Flash=pd.DataFrame(rmse_2017Flash)

In [ ]:
df_rmse_2017Flash

## **Flash Flood 2018**

we determine the water levels for Flash flood season 2018.

In [ ]:
WL2018Flash = testY1[-1310-78+365:-1310+365]
WL2018Flash

Now we predict the water levels for the 2018 Flash flood period using our trained models for different lead times and calculate the RMSE values for each prediction.

Finally we create a dataframe with the RMSE values for 2018 Flash flood

In [ ]:
predictANNY1_2018Flash=testpredictANN1[-1310-78+365:-1310+365].reshape(-1)
predictANNY3_2018Flash=testpredictANN3[-1310-78+365:-1310+365].reshape(-1)
predictANNY5_2018Flash=testpredictANN5[-1310-78+365:-1310+365].reshape(-1)
predictANNY7_2018Flash=testpredictANN7[-1310-78+365:-1310+365].reshape(-1)

rmseANN1day2018Flash= math.sqrt(mean_squared_error(predictANNY1_2018Flash,WL2018Flash))
print('RMSE ANN 1day 2018Flash = %.2f' %(rmseANN1day2018Flash))

rmseANN3day2018Flash= math.sqrt(mean_squared_error(predictANNY3_2018Flash,WL2018Flash))
print('RMSE ANN 3day 2018Flash = %.2f' %(rmseANN3day2018Flash))

rmseANN5day2018Flash= math.sqrt(mean_squared_error(predictANNY5_2018Flash,WL2018Flash))
print('RMSE ANN 5day 2018Flash = %.2f' %(rmseANN5day2018Flash))

rmseANN7day2018Flash= math.sqrt(mean_squared_error(predictANNY7_2018Flash,WL2018Flash))
print('RMSE ANN 7day 2018Flash = %.2f' %(rmseANN7day2018Flash))


predictRFY1_2018Flash=testpredictRF1[-1310-78+365:-1310+365].reshape(-1)
predictRFY3_2018Flash=testpredictRF3[-1310-78+365:-1310+365].reshape(-1)
predictRFY5_2018Flash=testpredictRF5[-1310-78+365:-1310+365].reshape(-1)
predictRFY7_2018Flash=testpredictRF7[-1310-78+365:-1310+365].reshape(-1)

rmseRF1day2018Flash= math.sqrt(mean_squared_error(predictRFY1_2018Flash,WL2018Flash))
print('RMSE RF 1day 2018Flash = %.2f' %(rmseRF1day2018Flash))

rmseRF3day2018Flash= math.sqrt(mean_squared_error(predictRFY3_2018Flash,WL2018Flash))
print('RMSE RF 3day 2018Flash = %.2f' %(rmseRF3day2018Flash))

rmseRF5day2018Flash= math.sqrt(mean_squared_error(predictRFY5_2018Flash,WL2018Flash))
print('RMSE RF 5day 2018Flash = %.2f' %(rmseRF5day2018Flash))

rmseRF7day2018Flash= math.sqrt(mean_squared_error(predictRFY7_2018Flash,WL2018Flash))
print('RMSE RF 7day 2018Flash = %.2f' %(rmseRF7day2018Flash))


predictCBY1_2018Flash=testpredictCB1[-1310-78+365:-1310+365].reshape(-1)
predictCBY3_2018Flash=testpredictCB3[-1310-78+365:-1310+365].reshape(-1)
predictCBY5_2018Flash=testpredictCB5[-1310-78+365:-1310+365].reshape(-1)
predictCBY7_2018Flash=testpredictCB7[-1310-78+365:-1310+365].reshape(-1)

rmseCB1day2018Flash= math.sqrt(mean_squared_error(predictCBY1_2018Flash,WL2018Flash))
print('RMSE CB 1day 2018Flash = %.2f' %(rmseCB1day2018Flash))

rmseCB3day2018Flash= math.sqrt(mean_squared_error(predictCBY3_2018Flash,WL2018Flash))
print('RMSE CB 3day 2018Flash = %.2f' %(rmseCB3day2018Flash))

rmseCB5day2018Flash= math.sqrt(mean_squared_error(predictCBY5_2018Flash,WL2018Flash))
print('RMSE CB 5day 2018Flash = %.2f' %(rmseCB5day2018Flash))

rmseCB7day2018Flash= math.sqrt(mean_squared_error(predictCBY7_2018Flash,WL2018Flash))
print('RMSE CB 7day 2018Flash = %.2f' %(rmseCB7day2018Flash))


predictLSTMY1_2018Flash=testpredictLSTM1[-1310-78+365:-1310+365].reshape(-1)
predictLSTMY3_2018Flash=testpredictLSTM3[-1310-78+365:-1310+365].reshape(-1)
predictLSTMY5_2018Flash=testpredictLSTM5[-1310-78+365:-1310+365].reshape(-1)
predictLSTMY7_2018Flash=testpredictLSTM7[-1310-78+365:-1310+365].reshape(-1)

rmseLSTM1day2018Flash= math.sqrt(mean_squared_error(predictLSTMY1_2018Flash,WL2018Flash))
print('RMSE LSTM 1day 2018Flash = %.2f' %(rmseLSTM1day2018Flash))

rmseLSTM3day2018Flash= math.sqrt(mean_squared_error(predictLSTMY3_2018Flash,WL2018Flash))
print('RMSE LSTM 3day 2018Flash = %.2f' %(rmseLSTM3day2018Flash))

rmseLSTM5day2018Flash= math.sqrt(mean_squared_error(predictLSTMY5_2018Flash,WL2018Flash))
print('RMSE LSTM 5day 2018Flash = %.2f' %(rmseLSTM5day2018Flash))

rmseLSTM7day2018Flash= math.sqrt(mean_squared_error(predictLSTMY7_2018Flash,WL2018Flash))
print('RMSE LSTM 7day 2018Flash = %.2f' %(rmseLSTM7day2018Flash))


rmse_2018Flash={'ANN':[rmseANN1day2018Flash,rmseANN3day2018Flash,rmseANN5day2018Flash,rmseANN7day2018Flash], 'RF':[rmseRF1day2018Flash,rmseRF3day2018Flash,rmseRF5day2018Flash,rmseRF7day2018Flash], 'CB':[rmseCB1day2018Flash,rmseCB3day2018Flash,rmseCB5day2018Flash,rmseCB7day2018Flash], 'LSTM': [rmseLSTM1day2018Flash,rmseLSTM3day2018Flash,rmseLSTM5day2018Flash,rmseLSTM7day2018Flash]}
df_rmse_2018Flash=pd.DataFrame(rmse_2018Flash)

In [ ]:
df_rmse_2018Flash

## **Flash Flood 2019**

we determine the water levels for Flash flood season 2019.

In [ ]:
WL2019Flash = testY1[-1310-78+365+365:-1310+365+365]
WL2019Flash

Now we predict the water levels for the 2019 Flash flood period using our trained models for different lead times and calculate the RMSE values for each prediction.

Finally we create a dataframe with the RMSE values for 2019 Flash flood

In [ ]:
predictANNY1_2019Flash=testpredictANN1[-1310-78+365+365:-1310+365+365].reshape(-1)
predictANNY3_2019Flash=testpredictANN3[-1310-78+365+365:-1310+365+365].reshape(-1)
predictANNY5_2019Flash=testpredictANN5[-1310-78+365+365:-1310+365+365].reshape(-1)
predictANNY7_2019Flash=testpredictANN7[-1310-78+365+365:-1310+365+365].reshape(-1)

rmseANN1day2019Flash= math.sqrt(mean_squared_error(predictANNY1_2019Flash,WL2019Flash))
print('RMSE ANN 1day 2019Flash = %.2f' %(rmseANN1day2019Flash))

rmseANN3day2019Flash= math.sqrt(mean_squared_error(predictANNY3_2019Flash,WL2019Flash))
print('RMSE ANN 3day 2019Flash = %.2f' %(rmseANN3day2019Flash))

rmseANN5day2019Flash= math.sqrt(mean_squared_error(predictANNY5_2019Flash,WL2019Flash))
print('RMSE ANN 5day 2019Flash = %.2f' %(rmseANN5day2019Flash))

rmseANN7day2019Flash= math.sqrt(mean_squared_error(predictANNY7_2019Flash,WL2019Flash))
print('RMSE ANN 7day 2019Flash = %.2f' %(rmseANN7day2019Flash))


predictRFY1_2019Flash=testpredictRF1[-1310-78+365+365:-1310+365+365].reshape(-1)
predictRFY3_2019Flash=testpredictRF3[-1310-78+365+365:-1310+365+365].reshape(-1)
predictRFY5_2019Flash=testpredictRF5[-1310-78+365+365:-1310+365+365].reshape(-1)
predictRFY7_2019Flash=testpredictRF7[-1310-78+365+365:-1310+365+365].reshape(-1)

rmseRF1day2019Flash= math.sqrt(mean_squared_error(predictRFY1_2019Flash,WL2019Flash))
print('RMSE RF 1day 2019Flash = %.2f' %(rmseRF1day2019Flash))

rmseRF3day2019Flash= math.sqrt(mean_squared_error(predictRFY3_2019Flash,WL2019Flash))
print('RMSE RF 3day 2019Flash = %.2f' %(rmseRF3day2019Flash))

rmseRF5day2019Flash= math.sqrt(mean_squared_error(predictRFY5_2019Flash,WL2019Flash))
print('RMSE RF 5day 2019Flash = %.2f' %(rmseRF5day2019Flash))

rmseRF7day2019Flash= math.sqrt(mean_squared_error(predictRFY7_2019Flash,WL2019Flash))
print('RMSE RF 7day 2019Flash = %.2f' %(rmseRF7day2019Flash))


predictCBY1_2019Flash=testpredictCB1[-1310-78+365+365:-1310+365+365].reshape(-1)
predictCBY3_2019Flash=testpredictCB3[-1310-78+365+365:-1310+365+365].reshape(-1)
predictCBY5_2019Flash=testpredictCB5[-1310-78+365+365:-1310+365+365].reshape(-1)
predictCBY7_2019Flash=testpredictCB7[-1310-78+365+365:-1310+365+365].reshape(-1)

rmseCB1day2019Flash= math.sqrt(mean_squared_error(predictCBY1_2019Flash,WL2019Flash))
print('RMSE CB 1day 2019Flash = %.2f' %(rmseCB1day2019Flash))

rmseCB3day2019Flash= math.sqrt(mean_squared_error(predictCBY3_2019Flash,WL2019Flash))
print('RMSE CB 3day 2019Flash = %.2f' %(rmseCB3day2019Flash))

rmseCB5day2019Flash= math.sqrt(mean_squared_error(predictCBY5_2019Flash,WL2019Flash))
print('RMSE CB 5day 2019Flash = %.2f' %(rmseCB5day2019Flash))

rmseCB7day2019Flash= math.sqrt(mean_squared_error(predictCBY7_2019Flash,WL2019Flash))
print('RMSE CB 7day 2019Flash = %.2f' %(rmseCB7day2019Flash))


predictLSTMY1_2019Flash=testpredictLSTM1[-1310-78+365+365:-1310+365+365].reshape(-1)
predictLSTMY3_2019Flash=testpredictLSTM3[-1310-78+365+365:-1310+365+365].reshape(-1)
predictLSTMY5_2019Flash=testpredictLSTM5[-1310-78+365+365:-1310+365+365].reshape(-1)
predictLSTMY7_2019Flash=testpredictLSTM7[-1310-78+365+365:-1310+365+365].reshape(-1)

rmseLSTM1day2019Flash= math.sqrt(mean_squared_error(predictLSTMY1_2019Flash,WL2019Flash))
print('RMSE LSTM 1day 2019Flash = %.2f' %(rmseLSTM1day2019Flash))

rmseLSTM3day2019Flash= math.sqrt(mean_squared_error(predictLSTMY3_2019Flash,WL2019Flash))
print('RMSE LSTM 3day 2019Flash = %.2f' %(rmseLSTM3day2019Flash))

rmseLSTM5day2019Flash= math.sqrt(mean_squared_error(predictLSTMY5_2019Flash,WL2019Flash))
print('RMSE LSTM 5day 2019Flash = %.2f' %(rmseLSTM5day2019Flash))

rmseLSTM7day2019Flash= math.sqrt(mean_squared_error(predictLSTMY7_2019Flash,WL2019Flash))
print('RMSE LSTM 7day 2019Flash = %.2f' %(rmseLSTM7day2019Flash))


rmse_2019Flash={'ANN':[rmseANN1day2019Flash,rmseANN3day2019Flash,rmseANN5day2019Flash,rmseANN7day2019Flash], 'RF':[rmseRF1day2019Flash,rmseRF3day2019Flash,rmseRF5day2019Flash,rmseRF7day2019Flash], 'CB':[rmseCB1day2019Flash,rmseCB3day2019Flash,rmseCB5day2019Flash,rmseCB7day2019Flash], 'LSTM': [rmseLSTM1day2019Flash,rmseLSTM3day2019Flash,rmseLSTM5day2019Flash,rmseLSTM7day2019Flash]}
df_rmse_2019Flash=pd.DataFrame(rmse_2019Flash)

In [ ]:
df_rmse_2019Flash

## **Flash Flood 2020**

we determine the water levels for Flash flood season 2020.

In [ ]:
WL2020Flash = testY1[-1310-78+366+365+365:-1310+366+365+365]
WL2020Flash

Now we predict the water levels for the 2020 Flash flood period using our trained models for different lead times and calculate the RMSE values for each prediction.

Finally we create a dataframe with the RMSE values for 2020 Flash flood

In [ ]:
predictANNY1_2020Flash=testpredictANN1[-1310-78+365+365+366:-1310+365+365+366].reshape(-1)
predictANNY3_2020Flash=testpredictANN3[-1310-78+365+365+366:-1310+365+365+366].reshape(-1)
predictANNY5_2020Flash=testpredictANN5[-1310-78+365+365+366:-1310+365+365+366].reshape(-1)
predictANNY7_2020Flash=testpredictANN7[-1310-78+365+365+366:-1310+365+365+366].reshape(-1)

rmseANN1day2020Flash= math.sqrt(mean_squared_error(predictANNY1_2020Flash,WL2020Flash))
print('RMSE ANN 1day 2020Flash = %.2f' %(rmseANN1day2020Flash))

rmseANN3day2020Flash= math.sqrt(mean_squared_error(predictANNY3_2020Flash,WL2020Flash))
print('RMSE ANN 3day 2020Flash = %.2f' %(rmseANN3day2020Flash))

rmseANN5day2020Flash= math.sqrt(mean_squared_error(predictANNY5_2020Flash,WL2020Flash))
print('RMSE ANN 5day 2020Flash = %.2f' %(rmseANN5day2020Flash))

rmseANN7day2020Flash= math.sqrt(mean_squared_error(predictANNY7_2020Flash,WL2020Flash))
print('RMSE ANN 7day 2020Flash = %.2f' %(rmseANN7day2020Flash))


predictRFY1_2020Flash=testpredictRF1[-1310-78+365+365+366:-1310+365+365+366].reshape(-1)
predictRFY3_2020Flash=testpredictRF3[-1310-78+365+365+366:-1310+365+365+366].reshape(-1)
predictRFY5_2020Flash=testpredictRF5[-1310-78+365+365+366:-1310+365+365+366].reshape(-1)
predictRFY7_2020Flash=testpredictRF7[-1310-78+365+365+366:-1310+365+365+366].reshape(-1)

rmseRF1day2020Flash= math.sqrt(mean_squared_error(predictRFY1_2020Flash,WL2020Flash))
print('RMSE RF 1day 2020Flash = %.2f' %(rmseRF1day2020Flash))

rmseRF3day2020Flash= math.sqrt(mean_squared_error(predictRFY3_2020Flash,WL2020Flash))
print('RMSE RF 3day 2020Flash = %.2f' %(rmseRF3day2020Flash))

rmseRF5day2020Flash= math.sqrt(mean_squared_error(predictRFY5_2020Flash,WL2020Flash))
print('RMSE RF 5day 2020Flash = %.2f' %(rmseRF5day2020Flash))

rmseRF7day2020Flash= math.sqrt(mean_squared_error(predictRFY7_2020Flash,WL2020Flash))
print('RMSE RF 7day 2020Flash = %.2f' %(rmseRF7day2020Flash))


predictCBY1_2020Flash=testpredictCB1[-1310-78+365+365+366:-1310+365+365+366].reshape(-1)
predictCBY3_2020Flash=testpredictCB3[-1310-78+365+365+366:-1310+365+365+366].reshape(-1)
predictCBY5_2020Flash=testpredictCB5[-1310-78+365+365+366:-1310+365+365+366].reshape(-1)
predictCBY7_2020Flash=testpredictCB7[-1310-78+365+365+366:-1310+365+365+366].reshape(-1)

rmseCB1day2020Flash= math.sqrt(mean_squared_error(predictCBY1_2020Flash,WL2020Flash))
print('RMSE CB 1day 2020Flash = %.2f' %(rmseCB1day2020Flash))

rmseCB3day2020Flash= math.sqrt(mean_squared_error(predictCBY3_2020Flash,WL2020Flash))
print('RMSE CB 3day 2020Flash = %.2f' %(rmseCB3day2020Flash))

rmseCB5day2020Flash= math.sqrt(mean_squared_error(predictCBY5_2020Flash,WL2020Flash))
print('RMSE CB 5day 2020Flash = %.2f' %(rmseCB5day2020Flash))

rmseCB7day2020Flash= math.sqrt(mean_squared_error(predictCBY7_2020Flash,WL2020Flash))
print('RMSE CB 7day 2020Flash = %.2f' %(rmseCB7day2020Flash))


predictLSTMY1_2020Flash=testpredictLSTM1[-1310-78+365+365+366:-1310+365+365+366].reshape(-1)
predictLSTMY3_2020Flash=testpredictLSTM3[-1310-78+365+365+366:-1310+365+365+366].reshape(-1)
predictLSTMY5_2020Flash=testpredictLSTM5[-1310-78+365+365+366:-1310+365+365+366].reshape(-1)
predictLSTMY7_2020Flash=testpredictLSTM7[-1310-78+365+365+366:-1310+365+365+366].reshape(-1)

rmseLSTM1day2020Flash= math.sqrt(mean_squared_error(predictLSTMY1_2020Flash,WL2020Flash))
print('RMSE LSTM 1day 2020Flash = %.2f' %(rmseLSTM1day2020Flash))

rmseLSTM3day2020Flash= math.sqrt(mean_squared_error(predictLSTMY3_2020Flash,WL2020Flash))
print('RMSE LSTM 3day 2020Flash = %.2f' %(rmseLSTM3day2020Flash))

rmseLSTM5day2020Flash= math.sqrt(mean_squared_error(predictLSTMY5_2020Flash,WL2020Flash))
print('RMSE LSTM 5day 2020Flash = %.2f' %(rmseLSTM5day2020Flash))

rmseLSTM7day2020Flash= math.sqrt(mean_squared_error(predictLSTMY7_2020Flash,WL2020Flash))
print('RMSE LSTM 7day 2020Flash = %.2f' %(rmseLSTM7day2020Flash))


rmse_2020Flash={'ANN':[rmseANN1day2020Flash,rmseANN3day2020Flash,rmseANN5day2020Flash,rmseANN7day2020Flash], 'RF':[rmseRF1day2020Flash,rmseRF3day2020Flash,rmseRF5day2020Flash,rmseRF7day2020Flash], 'CB':[rmseCB1day2020Flash,rmseCB3day2020Flash,rmseCB5day2020Flash,rmseCB7day2020Flash], 'LSTM': [rmseLSTM1day2020Flash,rmseLSTM3day2020Flash,rmseLSTM5day2020Flash,rmseLSTM7day2020Flash]}
df_rmse_2020Flash=pd.DataFrame(rmse_2020Flash)

In [ ]:
df_rmse_2020Flash

## **Flash Flood df**

Now we create a dataframe and merge all the RMSE values for the flash flood events.

In [ ]:
rmse_Flash = pd.concat([df_rmse_2017Flash, df_rmse_2018Flash, df_rmse_2019Flash, df_rmse_2020Flash ],axis=1)
rmse_Flash

# **Plots**

The following are codes for scatter plots for predicted vs actual water levels for different models at differnt lead times, followed by time series plots for Monsoon floods and Flash flood events for dofferent models at different leadtimes.

In [ ]:
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
from matplotlib.pyplot import figure
import os
import matplotlib.font_manager
dpi = 600
sizeX, sizeY = 6, 6
figure(figsize=(6, 6), dpi=600)

In [ ]:
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
from matplotlib.pyplot import figure
import os
import matplotlib.font_manager

for state in ['train','val','test']:
  for n in [1,3,5,7]:
    fig, ax = plt.subplots(2, 2)
    figure(figsize=(15, 15), dpi=600)
    for model in ['ANN','RF','CB','LSTM']:
      if model == 'ANN':
        color, ab,cd,axis, xlabel, ylabel= 'green',0,0,'on','','Predicted WL (m)'
      elif model == 'RF':
        color,ab,cd,axis, xlabel, ylabel = 'red',0,1,'on','',''
      elif model == 'CB':
        color,ab,cd ,axis, xlabel, ylabel= 'blue',1,0,'on','Observed WL (m)','Predicted WL (m)'
      elif model == 'LSTM':
        color,ab,cd,axis, xlabel, ylabel= 'magenta',1,1,'on','Observed WL (m)',''
      figname = "fig_"+state+model+str(n)
      axname = "ax_"+state+model+str(n)
      print(figname)
      print(axname)
      #figname, axname = plt.subplots()
      plt.style.use('seaborn-whitegrid')
      if model == 'LSTM':
        X=globals()[state+"Y_"+model+str(n)]
      else:
        X=globals()[state+"Y"+str(n)]
      
      Y=globals()[state+"predict"+model+str(n)]
      ax[ab,cd].scatter(X,Y, c=color, linewidth=1, edgecolor='black', alpha=0.5)


      ax[ab,cd].set_xlabel(xlabel, fontsize = 9)
      ax[ab,cd].set_ylabel(ylabel, fontsize = 9)

      ax[ab,cd].axis(axis)
      if model == 'ANN': ax[ab,cd].set_xticks([])
      elif model == 'RF': ax[ab,cd].set_xticks([])

      ax[ab,cd].plot([0,1],[0,1], transform=ax[ab,cd].transAxes, c='black', linewidth=0.5, alpha=0.5)

      plt.rcParams["font.family"] = "Times New Roman"
      plt.rcParams["font.size"] = 6
      
      if model == "ANN": modelname="DNN"
      else: modelname=model

      ax[ab,cd].set_title(state+" "+modelname+" Comb 1  T+"+str(n), fontsize = 9)

      #transform = axname.transAxes
      #line.set_transform(transform)
      #axname.add_line(line)

      rmse =globals() [state+'Score'+model+str(n)]
      r2= globals() [state+'r2_'+model+str(n)]
      #print(rmse)
      #print(r2)
      
      text1= "RMSE = "+str(float("{:.2f}".format(rmse)))+" (m)"
      text2= "R$^2$ = "+str(float("{:.3f}".format(r2)))

      ax[ab,cd].text(2.5,14,text1, fontsize = 9)
      ax[ab,cd].text(2.5,12,text2, fontsize = 9)

      ax[ab,cd].set_xlim([2, 16])
      ax[ab,cd].set_ylim([2, 16])

      figure(figsize=(6, 6), dpi=600)

    plt.tight_layout()
    #plt.subplots_adjust(wspace=0.4, hspace=1)
    plt.show()

    dir="/content/drive/MyDrive/Thesis/Plots/"+vars+"/Scatter_Plots/"+state+" Comb 1  T+"+str(n)+".jpg"
    print(dir)

    #fig.savefig(f"{dir}", dpi=600)

## **WL plots**

In [ ]:
dates = dataframe['Date']
dates_monsoon = dates[-1310:-1173]
dates_flash = dates[-1310-78:-1310]
dates_monsoon
dates_flash

### **Monsoon Plots**

In [ ]:
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
from matplotlib.pyplot import figure
import os
import matplotlib.font_manager

In [ ]:
for year in ['2017','2018','2019','2020']:
  fig,ax = plt.subplots(2,2)
  for n in [1,3,5,7]:
      if n==1: p,q,xlabel,ylabel=0,0,'',''
      if n==3: p,q,xlabel,ylabel=0,1,'',''
      if n==5: p,q,xlabel,ylabel=1,0,'Date','WL(m)'
      if n==7: p,q,xlabel,ylabel=1,1,'Date','WL(m)'
      figname= str(year)+" Monsoon Comb 1 T+"+str(n)
      print(figname)
      x= dates_monsoon
      for model in ['ANN','RF','CB','LSTM']:
          if model == 'ANN':
            color = 'green'
            modelname="DNN"
          elif model == 'RF':
            color = 'red'
            modelname=model
          elif model == 'CB':
            color = 'blue'
            modelname=model
          elif model == 'LSTM':
            color = 'magenta'
            modelname=model

          y = "predict"+model+"Y"+str(n)+"_"+str(year)+"Monsoon"
          print(y)
          ax[p,q].plot(dates_monsoon,(globals()[y]),linestyle='solid', linewidth =1, color=color, label = modelname)
        
          ax[p,q].set_xlabel('Date',fontsize=8)
          ax[p,q].set_ylabel('WL(m)',fontsize=8)
          ax[p,q].grid(False)
        

      ax[p,q].plot(dates_monsoon,globals()["WL"+str(year)+"Monsoon"],linestyle='solid', linewidth =2, color='black', label = 'Observed')
      ax[p,q].set_title(figname, fontsize=9)
      if n==1: ax[p,q].set_xticks([])
      elif n==3:ax[p,q].set_xticks([])
      start, end = ax[p,q].get_xlim()
      ax[p,q].xaxis.set_ticks(np.arange(start, end, 30))

      ax[p,q].legend(fontsize=6)
      plt.rcParams["font.family"] = "Times New Roman"
      plt.rcParams["font.size"] = 6
    
      plt.gcf().autofmt_xdate()
      ax[p,q].set_ylim(5,16)


  plt.tight_layout()
  
  dir="/content/drive/MyDrive/Thesis/Plots/"+vars+"/WL_Plots/"+figname+".jpg"
  print(dir)

  #plt.savefig(f"{dir}", dpi =600)
  plt.show()

  





### **Flash Plots**

In [ ]:
for year in ['2017','2018','2019','2020']:
  fig,ax = plt.subplots(2,2)
  for n in [1,3,5,7]:
      if n==1: p,q,xlabel,ylabel=0,0,'',''
      if n==3: p,q,xlabel,ylabel=0,1,'',''
      if n==5: p,q,xlabel,ylabel=1,0,'Date','WL(m)'
      if n==7: p,q,xlabel,ylabel=1,1,'Date','WL(m)'
      figname= str(year)+" Flash Comb 1 T+"+str(n)
      print(figname)
      x= dates_flash
      for model in ['ANN','RF','CB','LSTM']:
          if model == 'ANN':
            color = 'green'
            modelname="DNN"
          elif model == 'RF':
            color = 'red'
            modelname=model
          elif model == 'CB':
            color = 'blue'
            modelname=model
          elif model == 'LSTM':
            color = 'magenta'
            modelname=model

          y = "predict"+model+"Y"+str(n)+"_"+str(year)+"Flash"
          print(y)
          ax[p,q].plot(x,(globals()[y]),linestyle='solid', linewidth =1, color=color, label = modelname)
        
          ax[p,q].set_xlabel('Date',fontsize=8)
          ax[p,q].set_ylabel('WL(m)',fontsize=8)
          ax[p,q].grid(False)
        

      ax[p,q].plot(x,globals()["WL"+str(year)+"Flash"],linestyle='solid', linewidth =2, color='black', label = 'Observed')
      ax[p,q].set_title(figname, fontsize=9)
      if n==1: ax[p,q].set_xticks([])
      elif n==3:ax[p,q].set_xticks([])
      start, end = ax[p,q].get_xlim()
      ax[p,q].xaxis.set_ticks(np.arange(start, end, 15))

      ax[p,q].legend(fontsize=6)
      plt.rcParams["font.family"] = "Times New Roman"
      plt.rcParams["font.size"] = 6
    
      plt.gcf().autofmt_xdate()
      ax[p,q].set_ylim(2,16)


  
  dir="/content/drive/MyDrive/Thesis/Plots/"+vars+"/WL_Plots/"+figname+".jpg"
  print(dir)
  plt.tight_layout()

  #plt.savefig(f"{dir}", dpi=600)
  plt.show()



# **Finally, for a different combination of variables, change the value for comb at the begining and repeat the entire process. Thank you. ✌**